In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
df_train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
# null values
df_train.columns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
# column types
numerical_cols = df_train.select_dtypes(include=['int64', 'int32']).columns.tolist()
categorical_cols = df_train.select_dtypes(include=['object', 'category']).columns.tolist()

print("numerical Columns:", numerical_cols)
print("Categorical Columns:", categorical_cols)

In [ ]:
# # plot all the graphs and plots for the dataset to see how the data is organised

# # Histograms for numerical columns
# for col in numerical_cols:
#     plt.figure(figsize=(8, 6))
#     sns.histplot(data=df_train, x=col, kde=True)
#     plt.title(f'Histogram of {col}')
#     plt.show()

# # Box plots for numerical columns (useful for visualizing distribution and outliers)
# for col in numerical_cols:
#     plt.figure(figsize=(8, 6))
#     sns.boxplot(data=df_train, y=col)
#     plt.title(f'Box plot of {col}')
#     plt.show()

# # # Count plots for categorical columns
# # categorical_cols = df.select_dtypes(include='object').columns
# # for col in categorical_cols:
# #     plt.figure(figsize=(8, 6))
# #     sns.countplot(data=df, x=col)
# #     plt.title(f'Count plot of {col}')
# #     plt.xticks(rotation=45, ha='right')
# #     plt.show()

# # Pairplot (can be computationally intensive for large datasets)
# # sns.pairplot(df)
# # plt.title('Pairwise plots of numerical columns')
# # plt.show()

# # Correlation heatmap for numerical columns
# if len(numerical_cols) > 1:
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(df_train[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
#     plt.title('Correlation Heatmap of Numerical Columns')
#     plt.show()

# LightGBM + KMeans + Class Balancing (Optuna-tuned)

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# import optuna

# # Load data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Target mean encoding
# for col in ['Soil Type', 'Crop Type']:
#     mapping = train.groupby(col)['target'].mean()
#     train[f'{col}_enc'] = train[col].map(mapping)
#     test[f'{col}_enc'] = test[col].map(mapping)

# features = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous',
#             'Soil Type_enc','Crop Type_enc']
# X = train[features].copy()
# X_test = test[features].copy()

# # Add KMeans cluster
# kmeans = KMeans(n_clusters=5, random_state=42, n_init='auto')
# train['cluster'] = kmeans.fit_predict(X)
# test['cluster'] = kmeans.predict(X_test)
# X['cluster'] = train['cluster']
# X_test['cluster'] = test['cluster']
# y = train['target']

# # Class weights
# class_counts = y.value_counts().to_dict()
# total = len(y)
# weights = {cls: total / cnt for cls, cnt in class_counts.items()}
# sample_weights = y.map(weights)

# # Optuna objective
# def objective(trial):
#     param = {
#         'objective': 'multiclass',
#         'num_class': n_classes,
#         'metric': 'multi_logloss',
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('lr', 0.01, 0.3, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 32, 256),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
#     }
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     losses = []
#     for train_idx, val_idx in skf.split(X, y):
#         dtrain = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx], weight=sample_weights.iloc[train_idx])
#         dval = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])
#         gbm = lgb.train(
#             param, dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
#         )
#         preds = gbm.predict(X.iloc[val_idx])
#         losses.append(log_loss(y.iloc[val_idx], preds))
#     return np.mean(losses)

# # Optimize
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=40, timeout=600)
# best_params = study.best_params
# print("✅ Best Params:", best_params)


In [ ]:
# # Use optimized parameters
# best_params.update({
#     'objective': 'multiclass',
#     'num_class': n_classes,
#     'metric': 'multi_logloss',
#     'verbosity': -1,
#     'boosting_type': 'gbdt',
# })

# # Final bagged training
# from sklearn.model_selection import StratifiedKFold

# BAGS = 5
# skf = StratifiedKFold(n_splits=BAGS, shuffle=True, random_state=42)
# test_preds = np.zeros((X_test.shape[0], n_classes))

# for bag, (train_idx, val_idx) in enumerate(skf.split(X, y)):
#     print(f"Bag {bag+1}/{BAGS}")
#     dtrain = lgb.Dataset(X.iloc[train_idx], label=y.iloc[train_idx], weight=sample_weights.iloc[train_idx])
#     dval = lgb.Dataset(X.iloc[val_idx], label=y.iloc[val_idx])

#     gbm = lgb.train(
#         best_params,
#         dtrain,
#         num_boost_round=1000,
#         valid_sets=[dval],
#         callbacks=[
#             lgb.early_stopping(50),
#             lgb.log_evaluation(100)
#         ]
#     )

#     test_preds += gbm.predict(X_test) / BAGS

# # Convert predictions to top 3 labels
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# # Submission
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_lgb_kmeans_balance.csv', index=False)
# print("✅ submission_lgb_kmeans_balance.csv saved")


# LightGBM + KMeans cluster features + Class balancing (via sample weighting)

In [ ]:
# # ✅ Full Kaggle‑Compatible Model: LightGBM + KMeans Features + Class‑Balanced Bagging

# # 🧰 1. Imports
# import pandas as pd
# import numpy as np
# import lightgbm as lgb
# import optuna
# from sklearn.preprocessing import LabelEncoder, QuantileTransformer
# from sklearn.cluster import KMeans
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss

# # ───────────────────────────────────────────────────────────────────────────────
# # 🧱 2. Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Basic numeric features
# NUM = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# X = train[NUM].copy()
# X_test = test[NUM].copy()

# # Target‑mean encoding of categoricals
# for col in ['Soil Type','Crop Type']:
#     m = train.groupby(col)['target'].mean()
#     train[f'{col}_enc'] = train[col].map(m)
#     test[f'{col}_enc']  = test[col].map(m)

# X['soil_enc'] = train['Soil Type_enc']
# X['crop_enc'] = train['Crop Type_enc']
# X_test['soil_enc'] = test['Soil Type_enc']
# X_test['crop_enc'] = test['Crop Type_enc']

# # Quantile transform to BMI‑style distributed numeric inputs
# qt = QuantileTransformer(n_quantiles=1000, output_distribution='normal', random_state=42)
# X[NUM] = qt.fit_transform(X[NUM])
# X_test[NUM] = qt.transform(X_test[NUM])

# # KMeans clustering on numeric + encoded features
# kmeans = KMeans(n_clusters=20, random_state=42)
# X['km']      = kmeans.fit_predict(X)
# X_test['km'] = kmeans.predict(X_test)

# # Feature list
# features = X.columns.tolist()
# X = X.values
# X_test = X_test.values
# y = train['target'].values

# # ───────────────────────────────────────────────────────────────────────────────
# # 🔍 3. Optuna‑tuned LightGBM Objective
# def objective(trial):
#     params = {
#         'objective': 'multiclass', 'num_class': n_classes,
#         'metric': 'multi_logloss', 'verbosity': -1, 'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('lr', 0.02, 0.2, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 31, 256),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 5.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 5.0),
#     }
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     cv_losses = []
#     for tr_i, val_i in skf.split(X, y):
#         dtr = lgb.Dataset(X[tr_i], label=y[tr_i])
#         dval = lgb.Dataset(X[val_i], label=y[val_i])
#         gbm = lgb.train(params, dtr, num_boost_round=500,
#                         valid_sets=[dval],
#                         callbacks=[lgb.early_stopping(stopping_rounds=50)])
#         cv_losses.append(log_loss(y[val_i], gbm.predict(X[val_i])))
#     return np.mean(cv_losses)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=30, timeout=600)
# best = study.best_params
# best.update({'objective':'multiclass','num_class':n_classes, 'metric':'multi_logloss', 'verbosity':-1})
# print("OPTUNA BEST:", best)

# # ───────────────────────────────────────────────────────────────────────────────
# # 🎯 4. 5‑fold BAGGED LightGBM
# BAGS = 5
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# test_preds = np.zeros((len(test), n_classes))

# for bag in range(BAGS):
#     print(f"Bag {bag+1}/{BAGS}")
#     seed = 5000 + bag
#     bag_preds = np.zeros_like(test_preds)
#     for tr_i, val_i in skf.split(X, y):
#         dtr = lgb.Dataset(X[tr_i], label=y[tr_i])
#         gbm = lgb.train({**best, 'seed':seed},
#                         dtr, num_boost_round=best.get('num_boost_round',500))
#         bag_preds += gbm.predict(X_test) / skf.n_splits
#     test_preds += bag_preds / BAGS

# # ───────────────────────────────────────────────────────────────────────────────
# # 📝 5. Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)
# out = pd.DataFrame({'id': test['id'], 'Fertilizer Name': [' '.join(r) for r in labels]})
# out.to_csv('submission_lgbm_kmeans.csv', index=False)
# print("✅ Submission saved as submission_lgbm_kmeans.csv")


# Dynamic Cluster‑Based Blending,

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 Install dependencies (run once)
# !pip install lightgbm xgboost catboost scipy --quiet

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb
# from scipy.optimize import nnls

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Prep
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Raw numeric features for clustering
# NUM = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# X_raw = train[NUM].values
# X_test_raw = test[NUM].values

# # Scale before clustering
# scaler = StandardScaler().fit(np.vstack([X_raw, X_test_raw]))
# X_scaled = scaler.transform(X_raw)
# X_test_scaled = scaler.transform(X_test_raw)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Base‑Model OOF & Test Predictions
# N_SPLITS = 5
# skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# # placeholders
# oof_lgb = np.zeros((len(train), n_classes))
# oof_xgb = np.zeros_like(oof_lgb)
# oof_cb  = np.zeros_like(oof_lgb)

# test_lgb = np.zeros((len(test), n_classes))
# test_xgb = np.zeros_like(test_lgb)
# test_cb  = np.zeros_like(test_lgb)

# for fold, (tr, val) in enumerate(skf.split(X_scaled, train['target']), 1):
#     X_tr, X_val = X_scaled[tr], X_scaled[val]
#     y_tr, y_val = train['target'].iloc[tr], train['target'].iloc[val]

#     # LightGBM
#     m1 = lgb.LGBMClassifier(
#         objective='multiclass', num_class=n_classes,
#         learning_rate=0.05, n_estimators=500,
#         random_state=fold
#     )
#     m1.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
#            callbacks=[lgb.early_stopping(stopping_rounds=50)])
#     oof_lgb[val] = m1.predict_proba(X_val)
#     test_lgb  += m1.predict_proba(X_test_scaled) / N_SPLITS

#     # XGBoost
#     m2 = xgb.XGBClassifier(
#         objective='multi:softprob', num_class=n_classes,
#         learning_rate=0.05, n_estimators=500,
#         use_label_encoder=False, eval_metric='mlogloss',
#         random_state=fold
#     )
#     m2.fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
#            early_stopping_rounds=50, verbose=False)
#     oof_xgb[val] = m2.predict_proba(X_val)
#     test_xgb  += m2.predict_proba(X_test_scaled) / N_SPLITS

#     # CatBoost
#     m3 = cb.CatBoostClassifier(
#         iterations=500, learning_rate=0.05, depth=6,
#         loss_function='MultiClass', verbose=0, random_seed=fold
#     )
#     m3.fit(X_tr, y_tr)
#     oof_cb[val] = m3.predict_proba(X_val)
#     test_cb  += m3.predict_proba(X_test_scaled) / N_SPLITS

# # Stack OOF preds into [n_samples, 3 * n_classes]
# oof_stack = np.hstack([oof_lgb, oof_xgb, oof_cb])
# test_stack = np.hstack([test_lgb, test_xgb, test_cb])

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Cluster‑Based Weight Optimization
# K = 8  # number of clusters
# km = KMeans(n_clusters=K, random_state=42, n_init=10)
# clusters = km.fit_predict(X_scaled)
# test_clusters = km.predict(X_test_scaled)

# # Prepare container for cluster‑specific weights
# cluster_weights = np.zeros((K, 3))

# # For each cluster, solve non‑negative least squares to minimize log-loss
# for c in range(K):
#     idx = np.where(clusters == c)[0]
#     if len(idx) < n_classes:
#         # too few samples: fallback to uniform weights
#         cluster_weights[c] = [1/3, 1/3, 1/3]
#         continue

#     # build matrix A and vector b for nnls: 
#     # we’ll fit weights to approximate the one‑hot true labels
#     A = np.vstack([
#         oof_lgb[idx].flatten(),
#         oof_xgb[idx].flatten(),
#         oof_cb[idx].flatten()
#     ]).T.reshape(-1, 3)
#     # true occupancy vector b: for each sample and each class, 1 if true class, else 0
#     true = np.zeros((len(idx)*n_classes,))
#     for i, samp in enumerate(idx):
#         true[i*n_classes + train['target'].iloc[samp]] = 1.0

#     # solve nnls to get non‑negative weights w minimizing ||A w – true||²
#     w, _ = nnls(A, true)
#     cluster_weights[c] = w / w.sum()

# print("Cluster Weights:\n", cluster_weights)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Apply Cluster‑Specific Blending to Test Predictions
# final_test = np.zeros_like(test_lgb)
# for i in range(len(test)):
#     c = test_clusters[i]
#     w = cluster_weights[c]
#     p_l = test_lgb[i]
#     p_x = test_xgb[i]
#     p_c = test_cb[i]
#     final_test[i] = w[0]*p_l + w[1]*p_x + w[2]*p_c

# # ───────────────────────────────────────────────────────────────────────────────
# # 6️⃣ Build Final Submission
# top3 = np.argsort(final_test, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })

# submission.to_csv('submission_dynamic_cluster_blend.csv', index=False)
# print("✅ submission_dynamic_cluster_blend.csv saved!")


# One-hot encoding,Logistic regression model, Augmentation using the original dataset (

In [ ]:
# 1️⃣ Imports
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mutual_info_score, make_scorer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.base import BaseEstimator, ClassifierMixin, clone
from sklearn.utils.parallel import Parallel, delayed
from tqdm import tqdm

# 2️⃣ Load train + external original dataset
train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv', index_col='id')
test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv', index_col='id')
extra = pd.read_csv('/kaggle/input/fertilizer-prediction-df/Fertilizer Prediction.csv')
extra.columns = extra.columns.str.strip()

train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
test.rename(columns={'Temparature': 'Temperature'}, inplace=True)
extra.rename(columns={'Temparature': 'Temperature','Humidity ': 'Humidity'}, inplace=True)

# Encode target
le = LabelEncoder()
y = le.fit_transform(train.pop('Fertilizer Name'))
y_extra = le.transform(extra.pop('Fertilizer Name'))

# Treat all features as categorical
train = train.astype(str)
test = test.astype(str)
extra = extra.astype(str)
X_all = pd.concat([train, extra], axis=0).reset_index(drop=True)

# 3️⃣ Mutual Information based selection of 3-way combos
def adjusted_mutual_info(x, y, n_iter=5):
    x, y = x.astype(str), y.astype(str)
    m0 = mutual_info_score(x, y)
    m1 = Parallel(n_jobs=-1)(delayed(lambda rs: mutual_info_score(
        y, np.random.default_rng(rs).permutation(x)))(rs) for rs in range(n_iter))
    return m0 - np.mean(m1)

# Compute 3-way combos
e = mutual_info_score(y, y)
mi = {}
for c1, c2, c3 in tqdm(list(combinations(train.columns, 3))):
    col_name = f'{c1}_{c2}_{c3}'
    combined = X_all[c1] + '_' + X_all[c2] + '_' + X_all[c3]
    mi[col_name] = adjusted_mutual_info(combined, np.concatenate([y, y_extra])) / e

top_3_combos = sorted(mi, key=mi.get, reverse=True)[:10]  # Select top 10 combos

# 4️⃣ Create 2-combo + selected 3-combo features
def create_cross_features(df):
    df_new = df.copy()
    for c1, c2 in combinations(df.columns, 2):
        df_new[f'{c1}_{c2}'] = df[c1] + '_' + df[c2]
    for col in top_3_combos:
        c1, c2, c3 = col.split('_')
        df_new[col] = df[c1] + '_' + df[c2] + '_' + df[c3]
    return df_new

X_train_full = create_cross_features(train)
X_extra_full = create_cross_features(extra)
X_test_full  = create_cross_features(test)

# 5️⃣ Augmented Logistic Regression class
def Augmented(model, X_o, y_o, weight_arg='sample_weight', weight=1.0):
    class AugmentedModel(ClassifierMixin, BaseEstimator):
        def fit(self, X, y):
            sample_weight = np.array([1.0]*len(X)+[weight]*len(X_o))
            X_combined = pd.concat([X, X_o])
            y_combined = np.concatenate([y, y_o])
            self.m = clone(model).fit(X_combined, y_combined, **{weight_arg: sample_weight})
            self.classes_ = self.m.classes_
            return self
        def predict_proba(self, X):
            return self.m.predict_proba(X)
    return AugmentedModel()

# 6️⃣ MAP@3 scorer
def MAP(k):
    def mapk(y_true, y_pred):        
        y_pred = np.argsort(-y_pred, axis=1)[:, :k]
        m = (y_true[:, None] == y_pred)
        return np.mean(np.where(m.any(axis=1), 1/(m.argmax(axis=1)+1), 0))
    return mapk

MAP3 = MAP(3)
scorer = make_scorer(MAP3, response_method='predict_proba')

# 7️⃣ Train final model
model = Augmented(
    make_pipeline(
        OneHotEncoder(handle_unknown='ignore'),
        LogisticRegression(C=1e-2, max_iter=1000, random_state=0)
    ),
    X_extra_full, y_extra,
    weight_arg='logisticregression__sample_weight', weight=4.0
)

print("🧠 Training final model...")
model.fit(X_train_full, y)

# 8️⃣ Predict and prepare submission
probs = model.predict_proba(X_test_full)
top3 = np.argsort(-probs, axis=1)[:, :3]
labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

submission = pd.DataFrame({
    'id': test.index,
    'Fertilizer Name': [' '.join(row) for row in labels]
})
submission.to_csv("submission_logistic_ensemble.csv", index=False)
print("✅ Saved: submission_logistic_ensemble.csv")


In [ ]:
X_train_full.shape[1]

# CatBoost with cross-feature + bagging ensemble 

In [ ]:
# # ✅ CatBoost with feature interactions (CPU-compatible)

# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder

# # Load data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix column name typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Create cross feature
# train['Soil_Crop'] = train['Soil Type'] + "_" + train['Crop Type']
# test['Soil_Crop'] = test['Soil Type'] + "_" + test['Crop Type']

# # Categorical features
# cat_features = ['Soil Type', 'Crop Type', 'Soil_Crop']

# # Feature list
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous'] + cat_features
# X = train[features]
# y = train['target']
# X_test = test[features]

# # Stratified CV with CatBoost
# test_preds = np.zeros((X_test.shape[0], n_classes))
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
#     print(f"Training fold {fold + 1}/5")
    
#     X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
#     X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

#     train_pool = Pool(X_train, y_train, cat_features=cat_features)
#     val_pool = Pool(X_val, y_val, cat_features=cat_features)
#     test_pool = Pool(X_test, cat_features=cat_features)

#     model = CatBoostClassifier(
#         iterations=2000,
#         learning_rate=0.05,
#         depth=7,
#         eval_metric='MultiClass',
#         random_seed=42,
#         verbose=0,
#         early_stopping_rounds=50,
#         task_type='CPU'  # ✅ Use CPU to avoid CUDA error
#     )

#     model.fit(train_pool, eval_set=val_pool, use_best_model=True)
#     test_preds += model.predict_proba(test_pool) / skf.n_splits

# # Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })

# submission.to_csv('catboost_feature_interaction_cpu.csv', index=False)
# print("✅ Submission saved: catboost_feature_interaction_cpu.csv")


# LightGBM + KMeans Cluster Features + Optuna + Bagging

In [ ]:
# # 📦 Imports
# import pandas as pd
# import numpy as np
# import lightgbm as lgb
# import optuna
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# # 🧠 Load data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Target-mean encoding
# for col in ['Soil Type', 'Crop Type']:
#     m = train.groupby(col)['target'].mean()
#     train[f'{col}_enc'] = train[col].map(m)
#     test[f'{col}_enc'] = test[col].map(m)

# # Base features
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous',
#             'Soil Type_enc', 'Crop Type_enc']

# # Scale numeric features for clustering
# scaler = StandardScaler()
# X_for_cluster = scaler.fit_transform(train[features])

# # 🌀 Add KMeans cluster labels
# kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
# train['cluster'] = kmeans.fit_predict(X_for_cluster)
# test['cluster'] = kmeans.predict(scaler.transform(test[features]))

# # Add cluster to features
# features.append('cluster')

# X = train[features].values
# y = train['target'].values
# X_test = test[features].values

# # 🔍 Optuna tuning objective
# def objective(trial):
#     param = {
#         'objective': 'multiclass',
#         'num_class': n_classes,
#         'metric': 'multi_logloss',
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('lr', 0.01, 0.3, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 16, 256),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
#     }
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     losses = []
#     for train_idx, val_idx in skf.split(X, y):
#         dtrain = lgb.Dataset(X[train_idx], label=y[train_idx])
#         dval = lgb.Dataset(X[val_idx], label=y[val_idx])
#         gbm = lgb.train(
#             param, dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[
#                 lgb.early_stopping(stopping_rounds=50),
#                 lgb.log_evaluation(period=0)
#             ],
#         )
#         preds = gbm.predict(X[val_idx])
#         losses.append(log_loss(y[val_idx], preds))
#     return np.mean(losses)

# # 🔧 Run Optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=40, timeout=600)
# best_params = study.best_params
# print("Best params:", best_params)

# # 📦 Bagging + Final Predictions
# BAGS = 5
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# test_preds = np.zeros((X_test.shape[0], n_classes))

# for bag in range(BAGS):
#     print(f"Bag {bag + 1}/{BAGS}")
#     seed = 1000 + bag
#     preds_oof = np.zeros_like(test_preds)

#     for train_idx, val_idx in skf.split(X, y):
#         dtrain = lgb.Dataset(X[train_idx], label=y[train_idx])
#         dval = lgb.Dataset(X[val_idx], label=y[val_idx])

#         gbm = lgb.train(
#             {**best_params, 'seed': seed, 'objective': 'multiclass', 'num_class': n_classes, 'metric': 'multi_logloss'},
#             dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[lgb.early_stopping(50)],
#         )

#         preds_oof += gbm.predict(X_test) / skf.n_splits

#     test_preds += preds_oof / BAGS

# # 📝 Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_lgbm_clustered.csv', index=False)
# print("✅ submission_lgbm_clustered.csv saved")


# Stacking_LGB_XGB_CatB_NNMeta_v1

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 Install dependencies (run once)
# !pip install lightgbm xgboost catboost tensorflow --quiet

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Basic features (no heavy engineering for speed)
# features = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# X = train[features].values
# y = train['target'].values
# X_test = test[features].values

# # Scale features for meta‑model stability
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Generate Base‑Model OOF & Test Predictions
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Storage arrays
# oof_preds = np.zeros((X.shape[0], n_classes * 3))
# test_preds = np.zeros((X_test.shape[0], n_classes * 3))

# for fold, (tr, val) in enumerate(skf.split(X, y), 1):
#     print(f"▶ Fold {fold}/{n_splits}")
#     X_tr, X_val = X[tr], X[val]
#     y_tr, y_val = y[tr], y[val]

#     # — LightGBM
#     m1 = lgb.LGBMClassifier(
#         objective='multiclass', num_class=n_classes,
#         learning_rate=0.1, n_estimators=200, random_state=fold
#     )
#     m1.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50)])
#     oof_preds[val,   0*n_classes:1*n_classes] = m1.predict_proba(X_val)
#     test_preds[:,   0*n_classes:1*n_classes] += m1.predict_proba(X_test) / n_splits

#     # — XGBoost
#     m2 = xgb.XGBClassifier(
#         objective='multi:softprob', num_class=n_classes,
#         learning_rate=0.1, n_estimators=200,
#         use_label_encoder=False, eval_metric='mlogloss',
#         random_state=fold
#     )
#     m2.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
#     oof_preds[val,   1*n_classes:2*n_classes] = m2.predict_proba(X_val)
#     test_preds[:,   1*n_classes:2*n_classes] += m2.predict_proba(X_test) / n_splits

#     # — CatBoost
#     m3 = cb.CatBoostClassifier(
#         iterations=200, learning_rate=0.1, depth=6,
#         loss_function='MultiClass', verbose=0, random_seed=fold
#     )
#     m3.fit(X_tr, y_tr)
#     oof_preds[val,   2*n_classes:3*n_classes] = m3.predict_proba(X_val)
#     test_preds[:,   2*n_classes:3*n_classes] += m3.predict_proba(X_test) / n_splits

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Build & Train Meta‑Learner (Keras NN)
# # One‑hot encode the OOF targets
# y_ohe = tf.keras.utils.to_categorical(y, num_classes=n_classes)

# # Simple MLP
# meta = Sequential([
#     Dense(64, activation='relu', input_shape=(oof_preds.shape[1],)),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.2),
#     Dense(n_classes, activation='softmax')
# ])
# meta.compile(optimizer='adam', loss='categorical_crossentropy')

# # Early stopping on OOF loss
# es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# # Train on full OOF set
# meta.fit(oof_preds, y_ohe, epochs=50, batch_size=1024, callbacks=[es], verbose=2)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Final Predictions & Submission
# meta_probs = meta.predict(test_preds, batch_size=1024)
# top3 = np.argsort(meta_probs, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_stacked_nn_fast.csv', index=False)
# print("✅ submission_stacked_nn_fast.csv saved!")


# LightGBM, XGBoost, CatBoost (5-fold CV)

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 Install dependencies (run once)
# !pip install lightgbm xgboost catboost --quiet

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix column typo
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Target-mean encoding for categoricals
# for col in ['Soil Type','Crop Type']:
#     m = train.groupby(col)['target'].mean()
#     train[f'{col}_te'] = train[col].map(m)
#     test[f'{col}_te']  = test[col].map(m)

# # Feature list
# features = [
#     'Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous',
#     'Soil Type_te','Crop Type_te'
# ]

# X = train[features].values
# y = train['target'].values
# X_test = test[features].values

# # Scale features
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Prepare Out-of-Fold Predictions
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # We will store OOF and test predictions for each model
# oof_preds = np.zeros((X.shape[0], n_classes * 3))
# test_preds = np.zeros((X_test.shape[0], n_classes * 3))

# for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):
#     print(f"▶ Fold {fold}/{n_splits}")

#     X_tr, X_val = X[tr_idx], X[val_idx]
#     y_tr, y_val = y[tr_idx], y[val_idx]

#     # LightGBM
#     m_lgb = lgb.LGBMClassifier(
#         objective='multiclass', num_class=n_classes,
#         learning_rate=0.05, n_estimators=1000, random_state=fold
#     )
#     m_lgb.fit(
#         X_tr, y_tr,
#         eval_set=[(X_val, y_val)],
#         callbacks=[
#             lgb.early_stopping(stopping_rounds=50),
#             lgb.log_evaluation(period=0)
#         ]
#     )
#     oof_preds[val_idx, 0*n_classes:1*n_classes] = m_lgb.predict_proba(X_val)
#     test_preds[:,   0*n_classes:1*n_classes] += m_lgb.predict_proba(X_test) / n_splits

#     # XGBoost
#     m_xgb = xgb.XGBClassifier(
#         objective='multi:softprob', num_class=n_classes,
#         learning_rate=0.05, n_estimators=1000,
#         use_label_encoder=False, eval_metric='mlogloss',
#         random_state=fold
#     )
#     m_xgb.fit(
#         X_tr, y_tr,
#         eval_set=[(X_val, y_val)],
#         early_stopping_rounds=50,
#         verbose=False
#     )
#     oof_preds[val_idx, 1*n_classes:2*n_classes] = m_xgb.predict_proba(X_val)
#     test_preds[:,   1*n_classes:2*n_classes] += m_xgb.predict_proba(X_test) / n_splits

#     # CatBoost
#     m_cat = cb.CatBoostClassifier(
#         iterations=1000, learning_rate=0.05, depth=6,
#         loss_function='MultiClass', eval_metric='MultiClass',
#         verbose=0, random_seed=fold
#     )
#     m_cat.fit(X_tr, y_tr)
#     oof_preds[val_idx, 2*n_classes:3*n_classes] = m_cat.predict_proba(X_val)
#     test_preds[:,   2*n_classes:3*n_classes] += m_cat.predict_proba(X_test) / n_splits

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Train Meta-Model (Logistic Regression) on full OOF predictions
# meta = LogisticRegression(
#     multi_class='multinomial',
#     solver='lbfgs',
#     C=1.0,
#     max_iter=1000,
#     random_state=42
# )
# meta.fit(oof_preds, y)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Final Predict & Submission
# meta_probs = meta.predict_proba(test_preds)

# # Extract top-3 labels per sample
# top3 = np.argsort(meta_probs, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_stacked_lr.csv', index=False)
# print("✅ submission_stacked_lr.csv saved!")


# Stacking Ensemble (LGBM + CatBoost + XGBoost → Neural Net)

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 Install dependencies (run once)
# !pip install lightgbm xgboost catboost tensorflow --quiet

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.model_selection import StratifiedKFold
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Basic features (no heavy engineering for speed)
# features = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# X = train[features].values
# y = train['target'].values
# X_test = test[features].values

# # Scale features for meta‑model stability
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Generate Base‑Model OOF & Test Predictions
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# # Storage arrays
# oof_preds = np.zeros((X.shape[0], n_classes * 3))
# test_preds = np.zeros((X_test.shape[0], n_classes * 3))

# for fold, (tr, val) in enumerate(skf.split(X, y), 1):
#     print(f"▶ Fold {fold}/{n_splits}")
#     X_tr, X_val = X[tr], X[val]
#     y_tr, y_val = y[tr], y[val]

#     # — LightGBM
#     m1 = lgb.LGBMClassifier(
#         objective='multiclass', num_class=n_classes,
#         learning_rate=0.1, n_estimators=200, random_state=fold
#     )
#     m1.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(50)])
#     oof_preds[val,   0*n_classes:1*n_classes] = m1.predict_proba(X_val)
#     test_preds[:,   0*n_classes:1*n_classes] += m1.predict_proba(X_test) / n_splits

#     # — XGBoost
#     m2 = xgb.XGBClassifier(
#         objective='multi:softprob', num_class=n_classes,
#         learning_rate=0.1, n_estimators=200,
#         use_label_encoder=False, eval_metric='mlogloss',
#         random_state=fold
#     )
#     m2.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
#     oof_preds[val,   1*n_classes:2*n_classes] = m2.predict_proba(X_val)
#     test_preds[:,   1*n_classes:2*n_classes] += m2.predict_proba(X_test) / n_splits

#     # — CatBoost
#     m3 = cb.CatBoostClassifier(
#         iterations=200, learning_rate=0.1, depth=6,
#         loss_function='MultiClass', verbose=0, random_seed=fold
#     )
#     m3.fit(X_tr, y_tr)
#     oof_preds[val,   2*n_classes:3*n_classes] = m3.predict_proba(X_val)
#     test_preds[:,   2*n_classes:3*n_classes] += m3.predict_proba(X_test) / n_splits

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Build & Train Meta‑Learner (Keras NN)
# # One‑hot encode the OOF targets
# y_ohe = tf.keras.utils.to_categorical(y, num_classes=n_classes)

# # Simple MLP
# meta = Sequential([
#     Dense(64, activation='relu', input_shape=(oof_preds.shape[1],)),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.2),
#     Dense(n_classes, activation='softmax')
# ])
# meta.compile(optimizer='adam', loss='categorical_crossentropy')

# # Early stopping on OOF loss
# es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

# # Train on full OOF set
# meta.fit(oof_preds, y_ohe, epochs=100, batch_size=1024, callbacks=[es], verbose=2)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Final Predictions & Submission
# meta_probs = meta.predict(test_preds, batch_size=1024)
# top3 = np.argsort(meta_probs, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_stacked_nn_fast.csv', index=False)
# print("✅ submission_stacked_nn_fast.csv saved!")


# QuantileTransform + Smart Cross-Features 

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 Install dependencies (run once)
# !pip install lightgbm --quiet

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# import lightgbm as lgb
# from sklearn.preprocessing import LabelEncoder, QuantileTransformer
# from sklearn.model_selection import StratifiedKFold

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test .rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Smart Cross-Features

# # Ratio features
# train['N_P_ratio'] = train['Nitrogen'] / (train['Phosphorous'] + 1)
# test ['N_P_ratio'] = test ['Nitrogen'] / (test ['Phosphorous'] + 1)

# train['T_minus_H_over_M'] = (train['Temperature'] - train['Humidity']) / (train['Moisture'] + 1)
# test ['T_minus_H_over_M'] = (test ['Temperature'] - test ['Humidity']) / (test ['Moisture'] + 1)

# # Soil×Crop one-hot
# train['Soil_Crop'] = train['Soil Type'] + '_' + train['Crop Type']
# test ['Soil_Crop'] = test ['Soil Type'] + '_' + test ['Crop Type']
# dummies = pd.get_dummies(pd.concat([train['Soil_Crop'], test['Soil_Crop']]), prefix='SC')
# train_sc = dummies.iloc[:len(train)]
# test_sc  = dummies.iloc[len(train):].reset_index(drop=True)

# # Target‐mean encoding (for comparison)
# for col in ['Soil Type','Crop Type']:
#     m = train.groupby(col)['target'].mean()
#     train[f'{col}_te'] = train[col].map(m)
#     test [f'{col}_te'] = test [col].map(m)

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Assemble Feature Matrix

# # Numeric & ratio columns
# numeric_cols = [
#     'Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous',
#     'N_P_ratio','T_minus_H_over_M'
# ]
# # TE columns
# te_cols = ['Soil Type_te','Crop Type_te']

# # Combine
# X_num = train[numeric_cols + te_cols + ['target']].drop(columns='target')
# X_ratios = train[['N_P_ratio','T_minus_H_over_M']]
# X_num.reset_index(drop=True, inplace=True)
# X_train_full = pd.concat([X_num, train_sc.reset_index(drop=True)], axis=1)

# X_test_num = test[numeric_cols + te_cols]
# X_test_full = pd.concat([X_test_num.reset_index(drop=True), test_sc], axis=1)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Quantile Transform on numeric + ratio
# qt = QuantileTransformer(output_distribution='normal')
# qt_cols = numeric_cols + ['Soil Type_te','Crop Type_te','N_P_ratio','T_minus_H_over_M']
# X_train_full[qt_cols] = qt.fit_transform(X_train_full[qt_cols])
# X_test_full [qt_cols] = qt.transform(X_test_full [qt_cols])

# # Final feature matrices
# X = X_train_full.values
# X_test = X_test_full.values
# y = train['target'].values

# # ───────────────────────────────────────────────────────────────────────────────
# # 6️⃣ LightGBM Params (from Optuna best)
# params = {
#     'objective': 'multiclass',
#     'num_class': n_classes,
#     'metric': 'multi_logloss',
#     'boosting_type': 'gbdt',
#     'learning_rate': 0.0326699070625641,
#     'num_leaves': 225,
#     'min_data_in_leaf': 112,
#     'feature_fraction': 0.5899214712446607,
#     'bagging_fraction': 0.7276943630263858,
#     'bagging_freq': 2,
#     'lambda_l1': 2.8479755333998225,
#     'lambda_l2': 4.199355747289704,
#     'verbosity': -1
# }

# # ───────────────────────────────────────────────────────────────────────────────
# # 7️⃣ 5-Fold CV + Bagging Predictions
# BAGS = 5
# skf = StratifiedKFold(n_splits=BAGS, shuffle=True, random_state=42)
# test_preds = np.zeros((len(test), n_classes))

# for bag in range(BAGS):
#     print(f"Bag {bag+1}/{BAGS}")
#     seed = 2025 + bag
#     fold_preds = np.zeros_like(test_preds)

#     for train_idx, val_idx in skf.split(X, y):
#         X_tr, X_val = X[train_idx], X[val_idx]
#         y_tr, y_val = y[train_idx], y[val_idx]

#         dtrain = lgb.Dataset(X_tr, label=y_tr)
#         dval   = lgb.Dataset(X_val, label=y_val)
#         gbm = lgb.train(
#             {**params, 'seed': seed},
#             dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(period=0)]
#         )
#         fold_preds += gbm.predict(X_test) / skf.n_splits

#     test_preds += fold_preds / BAGS

# # ───────────────────────────────────────────────────────────────────────────────
# # 8️⃣ Make Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_lgbm_quantile_cross.csv', index=False)
# print("✅ submission_lgbm_quantile_cross.csv saved!")


# CatBoost + Bagging — strong baseline)

In [ ]:
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder

# # 2️⃣ Load & preprocess
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Target encoding
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Features
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous',
#             'Soil Type', 'Crop Type']
# cat_features = ['Soil Type', 'Crop Type']
# X = train[features]
# y = train['target']
# X_test = test[features]

# # 3️⃣ Bagging setup
# BAGS = 5
# skf = StratifiedKFold(n_splits=BAGS, shuffle=True, random_state=42)
# test_preds = np.zeros((len(test), n_classes))

# # 4️⃣ Train CatBoost
# for bag, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
#     print(f"Bag {bag}/{BAGS}")
#     X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     model = CatBoostClassifier(
#         iterations=1000,
#         learning_rate=0.05,
#         depth=6,
#         loss_function='MultiClass',
#         eval_metric='MultiClass',
#         verbose=0,
#         random_seed=42 + bag,
#         early_stopping_rounds=50
#     )

#     model.fit(
#         Pool(X_train, y_train, cat_features=cat_features),
#         eval_set=Pool(X_val, y_val, cat_features=cat_features)
#     )
#     preds = model.predict_proba(Pool(X_test, cat_features=cat_features))
#     test_preds += preds / BAGS

# # 5️⃣ Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_catboost_bag.csv', index=False)
# print("✅ submission_catboost_bag.csv saved")


# Pytourch based

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Install & Imports
# !pip install torch --quiet


In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from torch.utils.data import Dataset, DataLoader
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# df = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# df.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le_target = LabelEncoder()
# df['target'] = le_target.fit_transform(df['Fertilizer Name'])
# n_classes = df['target'].nunique()

# # Prepare features
# NUM_COLS = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# CAT_COLS = ['Soil Type','Crop Type']

# # Label-encode categoricals
# encoders = {}
# for col in CAT_COLS:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col])
#     test[col] = le.transform(test[col])
#     encoders[col] = le

# # Dataset split
# train_df, val_df = train_test_split(
#     df, test_size=0.1, stratify=df['target'], random_state=42
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Build PyTorch Dataset
# class TabularDataset(Dataset):
#     def __init__(self, df):
#         self.X_num = df[NUM_COLS].values.astype(np.float32)
#         self.X_cat = df[CAT_COLS].values.astype(np.int64)
#         self.y     = df['target'].values.astype(np.int64)
#     def __len__(self): return len(self.y)
#     def __getitem__(self, i):
#         return self.X_num[i], self.X_cat[i], self.y[i]

# train_ds = TabularDataset(train_df)
# val_ds   = TabularDataset(val_df)
# test_num = test[NUM_COLS].values.astype(np.float32)
# test_cat = test[CAT_COLS].values.astype(np.int64)

# train_loader = DataLoader(train_ds, batch_size=4096, shuffle=True, drop_last=True)
# val_loader   = DataLoader(val_ds,   batch_size=4096, shuffle=False)
# test_loader  = DataLoader(list(zip(test_num, test_cat)), batch_size=4096, shuffle=False)

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Define Model with Embeddings + MLP
# class EmbeddingMLP(nn.Module):
#     def __init__(self, num_cols, cat_dims, emb_dims, hidden_dims, n_out):
#         super().__init__()
#         # embeddings
#         self.embs = nn.ModuleList([
#             nn.Embedding(cat_dim, emb_dim)
#             for cat_dim, emb_dim in zip(cat_dims, emb_dims)
#         ])
#         input_dim = len(num_cols) + sum(emb_dims)
#         # MLP layers
#         layers = []
#         prev = input_dim
#         for h in hidden_dims:
#             layers += [nn.Linear(prev, h), nn.ReLU(), nn.BatchNorm1d(h), nn.Dropout(0.2)]
#             prev = h
#         layers.append(nn.Linear(prev, n_out))
#         self.net = nn.Sequential(*layers)

#     def forward(self, x_num, x_cat):
#         emb = [emb_layer(x_cat[:,i]) for i, emb_layer in enumerate(self.embs)]
#         x = torch.cat([x_num] + emb, dim=1)
#         return self.net(x)

# # Embedding sizes: take sqrt of cardinality (common heuristic)
# cat_dims = [df[col].nunique() for col in CAT_COLS]
# emb_dims = [min(50, (cd+1)//2) for cd in cat_dims]
# model = EmbeddingMLP(
#     num_cols=NUM_COLS,
#     cat_dims=cat_dims,
#     emb_dims=emb_dims,
#     hidden_dims=[256,128,64],
#     n_out=n_classes
# ).cuda()

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Train Loop
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# best_val_loss = np.inf
# patience, trials = 3, 0

# for epoch in range(20):
#     # Train
#     model.train()
#     for Xn, Xc, yb in train_loader:
#         Xn, Xc, yb = Xn.cuda(), Xc.cuda(), yb.cuda()
#         preds = model(Xn, Xc)
#         loss = criterion(preds, yb)
#         optimizer.zero_grad(); loss.backward(); optimizer.step()
#     # Validate
#     model.eval()
#     val_losses = []
#     with torch.no_grad():
#         for Xn, Xc, yb in val_loader:
#             Xn, Xc, yb = Xn.cuda(), Xc.cuda(), yb.cuda()
#             val_losses.append(criterion(model(Xn, Xc), yb).item())
#     val_loss = np.mean(val_losses)
#     print(f"Epoch {epoch+1}: val_loss={val_loss:.4f}")
#     # Early stopping
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss; torch.save(model.state_dict(), 'best_mlp.pt'); trials=0
#     else:
#         trials+=1
#         if trials>=patience: break

# # Load best
# model.load_state_dict(torch.load('best_mlp.pt'))

# # ───────────────────────────────────────────────────────────────────────────────
# # 6️⃣ Predict & Submission
# model.eval(); all_probs = []
# with torch.no_grad():
#     for Xn, Xc in test_loader:
#         Xn, Xc = Xn.cuda(), Xc.cuda()
#         all_probs.append(torch.softmax(model(Xn, Xc), dim=1).cpu().numpy())
# probs = np.vstack(all_probs)

# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# labels = le_target.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_mlp_embed.csv', index=False)
# print("✅ submission_mlp_embed.csv saved!")


# Wide & Deep in TensorFlow/Keras

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Install & Imports
# !pip install tensorflow --quiet

# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Label-encode target
# le_target = LabelEncoder()
# train['target'] = le_target.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Numeric features
# NUM_COLS = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']

# # Categorical features
# CAT_COLS = ['SoilType','CropType']
# # First, rename columns to remove spaces
# train.rename(columns={'Soil Type':'SoilType','Crop Type':'CropType'}, inplace=True)
# test.rename(columns={'Soil Type':'SoilType','Crop Type':'CropType'}, inplace=True)

# # Encode categories to integers
# for col in CAT_COLS:
#     enc = LabelEncoder()
#     train[col] = enc.fit_transform(train[col])
#     test[col]  = enc.transform(test[col])

# # Split train/val for early stopping
# X_train_df, X_val_df, y_train, y_val = train_test_split(
#     train, train['target'], stratify=train['target'], test_size=0.1, random_state=42
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Build Wide & Deep Model

# # Wide input: SoilType × CropType interaction
# interaction = Input(shape=(1,), name='soil_crop_inter', dtype='int32')

# # Deep numeric input
# num_input = Input(shape=(len(NUM_COLS),), name='num_input')

# # Deep categorical inputs & embeddings
# soil_in = Input(shape=(1,), name='SoilType_in', dtype='int32')
# soil_emb = Embedding(input_dim=train['SoilType'].nunique(),
#                      output_dim=min(50, train['SoilType'].nunique()//2),
#                      name='SoilType_emb')(soil_in)
# soil_emb_flat = Flatten()(soil_emb)

# crop_in = Input(shape=(1,), name='CropType_in', dtype='int32')
# crop_emb = Embedding(input_dim=train['CropType'].nunique(),
#                      output_dim=min(50, train['CropType'].nunique()//2),
#                      name='CropType_emb')(crop_in)
# crop_emb_flat = Flatten()(crop_emb)

# # Wide embedding layer acting as one-hot for interaction
# max_interact = train['SoilType'].nunique() * train['CropType'].nunique()
# wide_emb = Embedding(input_dim=max_interact,
#                      output_dim=max_interact,
#                      embeddings_initializer=tf.keras.initializers.Identity(),
#                      trainable=False,
#                      name='wide_emb')(interaction)
# wide_flat = Flatten()(wide_emb)

# # Deep tower
# deep_concat = Concatenate(name='deep_concat')([
#     num_input, soil_emb_flat, crop_emb_flat
# ])
# x = Dense(128, activation='relu')(deep_concat)
# x = Dense(64, activation='relu')(x)
# deep_out = Dense(32, activation='relu')(x)

# # Combine wide + deep
# combined = Concatenate(name='combined')([wide_flat, deep_out])
# output = Dense(n_classes, activation='softmax', name='predictions')(combined)

# model = Model(
#     inputs=[interaction, num_input, soil_in, crop_in],
#     outputs=output
# )
# model.compile(
#     optimizer=Adam(learning_rate=1e-3),
#     loss='sparse_categorical_crossentropy'
# )

# model.summary()

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Prepare Inputs

# def build_inputs(df):
#     return {
#         'soil_crop_inter': df['SoilType'] * train['CropType'].nunique() + df['CropType'],
#         'num_input': df[NUM_COLS].values,
#         'SoilType_in': df['SoilType'].values,
#         'CropType_in': df['CropType'].values,
#     }

# train_inputs = build_inputs(X_train_df)
# val_inputs   = build_inputs(X_val_df)
# test_inputs  = build_inputs(test)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Train

# callbacks = [
#     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# ]

# model.fit(
#     train_inputs, y_train.values,
#     validation_data=(val_inputs, y_val.values),
#     epochs=50, batch_size=2048,
#     callbacks=callbacks,
#     verbose=2
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 6️⃣ Predict & Create Submission

# probs = model.predict(test_inputs, batch_size=2048)
# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# labels = le_target.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_widedeep_corrected.csv', index=False)
# print("✅ submission_widedeep_corrected.csv saved!")


# TabPFN

In [ ]:
# pip install tabpfn


In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Install & Imports
# !pip install tabpfn --quiet

# import pandas as pd
# import numpy as np
# import torch
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from tabpfn import TabPFNClassifier

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & Preprocess Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Label‐encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])

# # Define features (you can swap in your engineered features here)
# features = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# X_full = train[features]
# y_full = train['target']
# X_test = test[features]

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Subsample for TabPFN (<=10 000 rows)
# SAMPLE_SIZE = 10000
# rng = np.random.RandomState(42)
# idx = rng.choice(len(X_full), size=SAMPLE_SIZE, replace=False)
# X_sub = X_full.iloc[idx].to_numpy()
# y_sub = y_full.iloc[idx].to_numpy()

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Train TabPFN
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model = TabPFNClassifier(device=device)
# model.fit(X_sub, y_sub)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Predict & Build Submission
# probs = model.predict_proba(X_test.to_numpy())  # shape (n_test, n_classes)
# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_tabpfn.csv', index=False)
# print("✅ submission_tabpfn.csv saved!")


# LightGBM and CatBoost mix

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 📦 1. Install dependencies (run this cell once at the top)
# !pip install lightgbm catboost --quiet

In [ ]:
# # ───────────────────────────────────────────────────────────────────────────────
# # 🔧 2. Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# from catboost import CatBoostClassifier, Pool

# # ───────────────────────────────────────────────────────────────────────────────
# # 📂 3. Load & Fix Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # ───────────────────────────────────────────────────────────────────────────────
# # 🧹 4. Smoothing-Regularized Target Encoding
# target = 'Fertilizer Name'
# le_target = LabelEncoder()
# train['target_enc'] = le_target.fit_transform(train[target])

# global_mean = train['target_enc'].mean()
# k_smooth = 30

# for col in ['Soil Type','Crop Type']:
#     stats = train.groupby(col)['target_enc'].agg(['mean','count'])
#     smooth = (stats['count'] * stats['mean'] + k_smooth * global_mean) / (stats['count'] + k_smooth)
#     train[f'{col}_te'] = train[col].map(smooth)
#     test[f'{col}_te']  = test[col].map(smooth)

# # ───────────────────────────────────────────────────────────────────────────────
# # 💡 5. Cross-Feature Interactions
# train['N_K_ratio']   = train['Nitrogen'] / (train['Potassium'] + 1)
# test['N_K_ratio']    = test['Nitrogen']  / (test['Potassium'] + 1)

# train['T_minus_H']   = train['Temperature'] - train['Humidity']
# test['T_minus_H']    = test['Temperature'] - test['Humidity']

# # label-encode Soil/Crop for interaction
# le_soil = LabelEncoder().fit(train['Soil Type'])
# le_crop = LabelEncoder().fit(train['Crop Type'])
# train['Soil_le'] = le_soil.transform(train['Soil Type'])
# train['Crop_le'] = le_crop.transform(train['Crop Type'])
# test['Soil_le']  = le_soil.transform(test['Soil Type'])
# test['Crop_le']  = le_crop.transform(test['Crop Type'])

# train['Soil_Crop_inter'] = train['Soil_le'] * 100 + train['Crop_le']
# test['Soil_Crop_inter']  = test['Soil_le']  * 100 + test['Crop_le']

# # ───────────────────────────────────────────────────────────────────────────────
# # 🗒️ 6. Prepare Feature Matrix
# feature_cols = [
#     'Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous',
#     'Soil Type_te','Crop Type_te',
#     'N_K_ratio','T_minus_H','Soil_Crop_inter'
# ]
# X = train[feature_cols]
# y = train['target_enc'].values
# X_test = test[feature_cols]

# # ───────────────────────────────────────────────────────────────────────────────
# # 🎯 7. Initial LightGBM Training (for pseudo-labels)
# X_tr, X_val, y_tr, y_val = train_test_split(
#     X, y, stratify=y, test_size=0.1, random_state=42
# )
# lgb_model = lgb.LGBMClassifier(
#     objective='multiclass',
#     num_class=len(le_target.classes_),
#     learning_rate=0.1,
#     n_estimators=500,
#     random_state=42,
#     verbose=-1
# )
# lgb_model.fit(
#     X_tr, y_tr,
#     eval_set=[(X_val, y_val)],
#     callbacks=[
#         lgb.early_stopping(stopping_rounds=50),
#         lgb.log_evaluation(period=100)  # change 100 to 0 if you want no logging
#     ]
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 🧪 8. Pseudo-Labeling High-Confidence Test Rows
# probs_test = lgb_model.predict_proba(X_test)
# conf_mask = (probs_test.max(axis=1) >= 0.80)
# X_pseudo = X_test[conf_mask]
# y_pseudo = np.argmax(probs_test[conf_mask], axis=1)

# # Combine original + pseudo
# X_ext = pd.concat([X, X_pseudo], ignore_index=True)
# y_ext = np.concatenate([y, y_pseudo])

# # ───────────────────────────────────────────────────────────────────────────────
# # 🏋️ 9. Retrain Models on Extended Data

# # 9.1 LightGBM on extended data
# lgb_final = lgb.LGBMClassifier(
#     objective='multiclass',
#     num_class=len(le_target.classes_),
#     learning_rate=0.05,
#     n_estimators=800,
#     random_state=2025,
#     verbose=-1
# )
# lgb_final.fit(
#     X_ext, y_ext,
#     eval_set=[(X_ext, y_ext)],
#      callbacks=[
#         lgb.early_stopping(stopping_rounds=50),
#     ]
# )

# # 9.2 CatBoost on extended data
# cat_model = CatBoostClassifier(
#     iterations=800,
#     learning_rate=0.05,
#     depth=6,
#     loss_function='MultiClass',
#     eval_metric='MultiClass',
#     random_seed=2025,
#     verbose=100
# )
# cat_model.fit(
#     X_ext, y_ext,
#     cat_features=[]  # all features are numeric/encoded
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 🔄 10. Blend & Predict Top-3
# proba_lgb = lgb_final.predict_proba(X_test)
# proba_cat = cat_model.predict_proba(X_test)
# final_proba = 0.7 * proba_lgb + 0.3 * proba_cat

# top3_idx    = np.argsort(final_proba, axis=1)[:, -3:][:, ::-1]
# top3_labels = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# # ───────────────────────────────────────────────────────────────────────────────
# # 📤 11. Submission
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })
# submission.to_csv('submission_pseudo_sm_te_inter_blend.csv', index=False)
# print("✅ submission_pseudo_sm_te_inter_blend.csv saved!")


# Hybrid TabNet + LightGBM Ensemble with Stratified K-Fold

In [ ]:
# # Let's generate the complete corrected code for the hybrid TabNet + LightGBM ensemble
# # including installation, preprocessing, training, and prediction in a Kaggle-compatible way.

# # 📌 1. Imports
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# from pytorch_tabnet.tab_model import TabNetClassifier
# import torch
# import warnings
# from lightgbm import early_stopping, log_evaluation

# warnings.filterwarnings("ignore")

# # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# # 📌 2. Load & Preprocess Data
# train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
# test  = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")

# # Fix typo
# train.rename(columns={"Temparature": "Temperature"}, inplace=True)
# test.rename(columns={"Temparature": "Temperature"}, inplace=True)

# # Label encode target
# le = LabelEncoder()
# train["target"] = le.fit_transform(train["Fertilizer Name"])
# target = "target"

# # Feature columns
# cat_cols = ["Soil Type", "Crop Type"]
# num_cols = ["Temperature", "Humidity", "Moisture", "Nitrogen", "Potassium", "Phosphorous"]
# features = num_cols + cat_cols

# # Combine data for consistent encoding
# train["is_train"] = 1
# test["is_train"] = 0
# full_df = pd.concat([train[features + ["is_train"]], test[features + ["is_train"]]])

# # Encode categoricals
# for col in cat_cols:
#     le_col = LabelEncoder()
#     full_df[col] = le_col.fit_transform(full_df[col])

# # Split back
# train_df = full_df[full_df.is_train == 1].drop("is_train", axis=1).copy()
# test_df  = full_df[full_df.is_train == 0].drop("is_train", axis=1).copy()
# X = train_df[features]
# y = train[target]
# X_test = test_df[features]
# n_classes = y.nunique()

# # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# # 📌 3. Training with Stratified K-Fold (TabNet + LGB)
# N_SPLITS = 5
# skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# oof_tabnet = np.zeros((len(X), n_classes))
# oof_lgb    = np.zeros((len(X), n_classes))
# preds_tabnet = np.zeros((len(X_test), n_classes))
# preds_lgb    = np.zeros((len(X_test), n_classes))

# for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
#     print(f"🔁 Fold {fold}")
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # ➤ LightGBM
#     lgb_model = lgb.LGBMClassifier(
#         objective="multiclass",
#         num_class=n_classes,
#         learning_rate=0.05,
#         n_estimators=500,
#         random_state=fold,
#         force_row_wise=True  # efficient row-wise processing
#     )
#     lgb_model.fit(
#     X_tr, y_tr,
#     eval_set=[(X_val, y_val)],
#     callbacks=[
#         early_stopping(stopping_rounds=50),
#         log_evaluation(period=0)  # This replaces the need for verbose
#     ]
# )
#     oof_lgb[val_idx] = lgb_model.predict_proba(X_val)
#     preds_lgb += lgb_model.predict_proba(X_test) / N_SPLITS

#     # ➤ TabNet
#     tabnet = TabNetClassifier(
#         n_d=16, n_a=16,
#         n_steps=5,
#         gamma=1.5,
#         n_independent=2,
#         n_shared=2,
#         optimizer_fn=torch.optim.Adam,
#         optimizer_params=dict(lr=2e-2),
#         mask_type='entmax',
#         verbose=0,
#         seed=fold
#     )
#     tabnet.fit(
#         X_tr.values, y_tr.values,
#         eval_set=[(X_val.values, y_val.values)],
#         eval_metric=["logloss"],
#         max_epochs=200,
#         patience=20,
#         batch_size=1024,
#         virtual_batch_size=128,
#         num_workers=0,
#         drop_last=False
#     )
#     oof_tabnet[val_idx] = tabnet.predict_proba(X_val.values)
#     preds_tabnet += tabnet.predict_proba(X_test.values) / N_SPLITS

# # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# # 📌 4. Blending Predictions
# oof_blend = 0.6 * oof_lgb + 0.4 * oof_tabnet
# final_preds = 0.6 * preds_lgb + 0.4 * preds_tabnet
# print("📉 OOF LogLoss:", log_loss(y, oof_blend))

# # ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
# # 📌 5. Prepare Submission
# top3 = np.argsort(final_preds, axis=1)[:, -3:][:, ::-1]
# top3_labels = le.inverse_transform(top3.ravel()).reshape(-1, 3)
# submission = pd.DataFrame({
#     "id": test["id"],
#     "Fertilizer Name": [" ".join(row) for row in top3_labels]
# })
# submission.to_csv("submission_tabnet_lgb.csv", index=False)
# print("✅ submission_tabnet_lgb.csv saved.")



In [ ]:
# # Install dependencies (run this cell once at the top)
# !pip install lightgbm --quiet
# !pip install pytorch-tabnet --quiet

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import log_loss
# from lightgbm import LGBMClassifier, early_stopping, log_evaluation
# from pytorch_tabnet.tab_model import TabNetClassifier
# import torch

# # 1️⃣ Load & Prepare Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# sub = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')

# # Fix typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode Target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Encode categorical features
# cat_cols = ['Soil Type', 'Crop Type']
# for col in cat_cols:
#     le_col = LabelEncoder()
#     train[col] = le_col.fit_transform(train[col])
#     test[col] = le_col.transform(test[col])

# # Feature list
# features = ['Soil Type', 'Crop Type', 'Temperature', 'Humidity', 
#             'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']

# X = train[features].copy()
# y = train['target'].copy()
# X_test = test[features].copy()

# # 2️⃣ TabNet + LightGBM Ensemble
# N_SPLITS = 5
# oof_tabnet = np.zeros((len(train), n_classes))
# oof_lgb = np.zeros((len(train), n_classes))
# preds_tabnet = np.zeros((len(test), n_classes))
# preds_lgb = np.zeros((len(test), n_classes))

# skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y)):
#     print(f"📁 Fold {fold + 1}")

#     X_tr, X_val = X.iloc[tr_idx].values, X.iloc[val_idx].values
#     y_tr, y_val = y.iloc[tr_idx].values, y.iloc[val_idx].values

#     # ➤ TabNet
#     tabnet = TabNetClassifier(
#         seed=42,
#         verbose=0,
#         device_name='cuda' if torch.cuda.is_available() else 'cpu'
#     )
#     tabnet.fit(
#         X_tr, y_tr,
#         eval_set=[(X_val, y_val)],
#         eval_name=["val"],
#         eval_metric=["logloss"],
#         patience=20,
#         max_epochs=200,
#         batch_size=1024,
#         virtual_batch_size=128
#     )
#     oof_tabnet[val_idx] = tabnet.predict_proba(X_val)
#     preds_tabnet += tabnet.predict_proba(X_test.values) / N_SPLITS

#     # ➤ LightGBM
#     lgb_model = LGBMClassifier(
#         objective="multiclass",
#         num_class=n_classes,
#         learning_rate=0.05,
#         n_estimators=500,
#         random_state=fold,
#         force_row_wise=True
#     )
#     lgb_model.fit(
#         X.iloc[tr_idx], y.iloc[tr_idx],
#         eval_set=[(X.iloc[val_idx], y.iloc[val_idx])],
#         callbacks=[
#             early_stopping(stopping_rounds=50),
#             log_evaluation(period=0)
#         ]
#     )
#     oof_lgb[val_idx] = lgb_model.predict_proba(X.iloc[val_idx])
#     preds_lgb += lgb_model.predict_proba(X_test) / N_SPLITS

# # 3️⃣ Ensemble Predictions
# # Weighted average (adjust these if desired)
# final_preds = 0.55 * preds_tabnet + 0.45 * preds_lgb

# # 4️⃣ Create Submission File
# top3 = np.argsort(final_preds, axis=1)[:, -3:][:, ::-1]
# top3_labels = le.inverse_transform(top3.ravel()).reshape(-1, 3)
# sub['Fertilizer Name'] = [' '.join(row) for row in top3_labels]
# sub.to_csv("submission_tabnet_lgbm_ensemble.csv", index=False)
# print("✅ submission_tabnet_lgbm_ensemble.csv saved!")


# H2O method

In [ ]:
# # 📦 1. Install and import
# !pip install -q h2o

In [ ]:
# import h2o
# from h2o.automl import H2OAutoML
# import pandas as pd
# import numpy as np

# # 🔄 2. Initialize H2O
# h2o.init()

# # 📂 3. Load Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # 🧼 4. Fix column typo (just in case)
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # 🏷️ 5. Target Encoding
# target = 'Fertilizer Name'
# id_col = 'id'
# features = [col for col in train.columns if col not in [target, id_col]]

# # Convert to H2OFrames
# train_h2o = h2o.H2OFrame(train[features + [target]])
# test_h2o = h2o.H2OFrame(test[features])

# # 🔮 6. Run AutoML
# aml = H2OAutoML(max_models=20, seed=42, sort_metric='mean_per_class_error')
# aml.train(x=features, y=target, training_frame=train_h2o)

# # ✅ 7. Predict on test
# preds = aml.leader.predict(test_h2o)
# preds_df = preds.as_data_frame()

# # 🥇 8. Create submission with top 3 predictions
# top3 = preds_df[['predict', 'p1', 'p2', 'p3']]
# top3_preds = preds_df.drop('predict', axis=1).apply(
#     lambda row: ' '.join(preds.columns[1:][row.values.argsort()[::-1][:3]]), axis=1)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': top3_preds
# })
# submission.to_csv('submission_h2o.csv', index=False)
# print("✅ submission_h2o.csv saved.")


# CatBoost-based pipeline

In [ ]:
# # 📦 1. Imports
# import pandas as pd
# import numpy as np
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import train_test_split

# # 📂 2. Load Data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # 🔧 3. Target & Categorical Columns
# target_col = "Fertilizer Name"
# cat_cols = ['Soil Type', 'Crop Type']
# num_cols = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
# features = num_cols + cat_cols

# # 🧪 4. Split for validation (optional, for local tuning)
# X_train, X_val, y_train, y_val = train_test_split(
#     train[features], train[target_col], test_size=0.1, random_state=42, stratify=train[target_col]
# )

# # 🧠 5. CatBoost Model Setup
# model = CatBoostClassifier(
#     iterations=3000,
#     learning_rate=0.03,
#     depth=7,
#     eval_metric='MultiClass',
#     loss_function='MultiClass',
#     cat_features=cat_cols,
#     verbose=200,
#     early_stopping_rounds=100,
#     random_state=42
# )

# # 🏋️ 6. Train Model
# train_pool = Pool(X_train, y_train, cat_features=cat_cols)
# val_pool = Pool(X_val, y_val, cat_features=cat_cols)
# model.fit(train_pool, eval_set=val_pool)

# # 📊 7. Predict Top-3 Labels
# test_pool = Pool(test[features], cat_features=cat_cols)
# probs = model.predict_proba(test_pool)
# labels = model.classes_

# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# top3_labels = labels[top3]

# # 📝 8. Submission Format
# submission = pd.DataFrame({
#     "id": test["id"],
#     "Fertilizer Name": [' '.join(row) for row in top3_labels]
# })

# submission.to_csv("submission_catboost.csv", index=False)
# print("✅ submission_catboost.csv saved!")


## another catboots based model

In [ ]:
# # 📦 Install CatBoost if needed (run once at top)
# # !pip install catboost --quiet

In [ ]:


# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from catboost import CatBoostClassifier, Pool

# # ─────────────────────────────────────────────────────────
# # 1️⃣ Load & Prepare Data
# df = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv', index_col='id')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# df.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Separate X and y
# X = df.drop(columns=['Fertilizer Name'])
# y = df['Fertilizer Name']

# # Label-encode the target
# le_target = LabelEncoder()
# y_enc = le_target.fit_transform(y)

# # Features
# cat_cols = ['Soil Type','Crop Type']
# num_cols = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# features = num_cols + cat_cols

# # We'll train on DataFrames for CatBoost
# X = X[features].copy()
# test = test[features].copy()

# # ─────────────────────────────────────────────────────────
# # helper: MAP@3
# def mapk(actual, predicted, k=3):
#     def apk(a, p):
#         p = p[:k]
#         score = 0.0
#         hits = 0
#         seen = set()
#         for i, pred in enumerate(p):
#             if pred in a and pred not in seen:
#                 hits += 1
#                 score += hits / (i+1.0)
#                 seen.add(pred)
#         return score / min(len(a), k)
#     return np.mean([apk(a, p) for a, p in zip(actual, predicted)])

# # ─────────────────────────────────────────────────────────
# # 2️⃣ Try different expansion counts
# results = []
# ocount = 0  # no external data in this run

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for count in range(1, 6):  # try counts 1 through 5
#     print(f"\n▶▶▶ Expansion count = {count}")
#     fold_scores = []
    
#     # prepare external placeholders (empty)
#     xo = pd.DataFrame(columns=features)
#     yo = np.array([])

#     # OOF predictions just to compute MAP@3
#     oof_preds = np.zeros((len(X), 3), dtype=object)  # store top3 lists

#     for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y_enc), 1):
#         # split
#         X_tr = X.iloc[tr_idx].reset_index(drop=True)
#         y_tr = pd.Series(y_enc[tr_idx])
#         X_val = X.iloc[val_idx]
#         y_val = y_enc[val_idx]

#         # 2.1 expand
#         X_tr_ext = pd.concat([X_tr]*count + [xo]*ocount, ignore_index=True)
#         y_tr_ext = pd.concat([y_tr]*count + [pd.Series(yo)*ocount], ignore_index=True)

#         # 2.2 train CatBoost
#         model = CatBoostClassifier(
#             iterations=500,
#             learning_rate=0.1,
#             depth=6,
#             loss_function='MultiClass',
#             eval_metric='MultiClass',
#             cat_features=[X_tr_ext.columns.get_loc(c) for c in cat_cols],
#             task_type='CPU', verbose=0
#         )
#         model.fit(X_tr_ext, y_tr_ext)

#         # 2.3 predict val
#         probas = model.predict_proba(X_val)
#         top3_idx = np.argsort(probas, axis=1)[:, -3:][:, ::-1]
#         top3_labels = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

#         # compute MAP@3 on this fold
#         actual = [[lab] for lab in y_val]
#         fold_score = mapk(actual, top3_labels.tolist(), k=3)
#         fold_scores.append(fold_score)

#         print(f"  Fold {fold} MAP@3 = {fold_score:.4f}")

#     mean_score = np.mean(fold_scores)
#     print(f"→ Expansion count {count} mean MAP@3 = {mean_score:.4f}")
#     results.append((count, mean_score))

# # ─────────────────────────────────────────────────────────
# # 3️⃣ Show best count
# best_count, best_score = max(results, key=lambda x: x[1])
# print(f"\n🎉 Best expansion count = {best_count} with MAP@3 = {best_score:.4f}")

# # ─────────────────────────────────────────────────────────
# # 4️⃣ Train final model on full expanded set and predict test

# # Expand full training set
# X_full_ext = pd.concat([X]*best_count, ignore_index=True)
# y_full_ext = np.tile(y_enc, best_count)

# # Train final
# final_model = CatBoostClassifier(
#     iterations=500,
#     learning_rate=0.1,
#     depth=6,
#     loss_function='MultiClass',
#     eval_metric='MultiClass',
#     cat_features=[X_full_ext.columns.get_loc(c) for c in cat_cols],
#     task_type='CPU',
#     verbose=100
# )
# final_model.fit(X_full_ext, y_full_ext)

# # Predict test
# probas_test = final_model.predict_proba(test)
# top3_idx = np.argsort(probas_test, axis=1)[:, -3:][:, ::-1]
# top3_labels = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# # Submission
# submission = pd.DataFrame({
#     'id': test.index,
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })
# submission.to_csv('submission_expansion.csv', index=False)
# print("✅ submission_expansion.csv saved!")


# LightGBM-only pipeline

In [ ]:
!pip install lightgbm --quiet


In [ ]:
# # LightGBM-Only Fast Baseline for Fertilizer Recommendation (Kaggle Playground S5E6)

# import pandas as pd
# import lightgbm as lgb
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from lightgbm import LGBMClassifier
# import numpy as np

# # Load data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix column name typo if exists
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = len(le.classes_)

# # Target mean encoding for categoricals
# for col in ['Soil Type', 'Crop Type']:
#     means = train.groupby(col)['target'].mean()
#     train[f'{col}_enc'] = train[col].map(means)
#     test[f'{col}_enc'] = test[col].map(means)

# # Define features
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous',
#             'Soil Type_enc', 'Crop Type_enc']
# X = train[features]
# y = train['target']
# X_test = test[features]

# # Train/validation split
# X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# # LightGBM dataset
# train_set = lgb.Dataset(X_train, label=y_train)
# val_set = lgb.Dataset(X_val, label=y_val)

# # LightGBM params
# params = {
#     'objective': 'multiclass',
#     'num_class': n_classes,
#     'metric': 'multi_logloss',
#     'learning_rate': 0.1,
#     'verbosity': -1,
#     'seed': 42
# }

# callbacks = [
#     lgb.early_stopping(stopping_rounds=50),
#     lgb.log_evaluation(period=100)
# ]

# model = lgb.train(
#     params,
#     train_set,
#     num_boost_round=10000,
#     valid_sets=[train_set, val_set],
#     callbacks=callbacks
# )

# # Predict
# probs = model.predict(X_test)
# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# top3_labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)

# # Build submission
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })
# submission.to_csv('submission_lgbm_fast.csv', index=False)

# print("✅ submission_lgbm_fast.csv saved.")


# LightGBM + Bagging

In [ ]:
# # 📦 Install dependencies (run once at top)
# !pip install lightgbm optuna --quiet

# # ───────────────────────────────────────────────────────────────────────────────

In [ ]:
# # 1️⃣ Imports
# import pandas as pd
# import numpy as np
# import lightgbm as lgb
# import optuna
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Load & preprocess data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# train['target'] = le.fit_transform(train['Fertilizer Name'])
# n_classes = train['target'].nunique()

# # Target‐mean encoding for categoricals
# for col in ['Soil Type','Crop Type']:
#     m = train.groupby(col)['target'].mean()
#     train[f'{col}_enc'] = train[col].map(m)
#     test[f'{col}_enc']  = test[col].map(m)

# # Features
# features = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous',
#             'Soil Type_enc','Crop Type_enc']
# X = train[features].values
# y = train['target'].values
# X_test = test[features].values

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Optuna objective to tune LightGBM
# def objective(trial):
#     param = {
#         'objective': 'multiclass',
#         'num_class': n_classes,
#         'metric': 'multi_logloss',
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'learning_rate': trial.suggest_float('lr', 0.01, 0.3, log=True),
#         'num_leaves': trial.suggest_int('num_leaves', 16, 256),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 200),
#         'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
#         'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
#         'lambda_l1': trial.suggest_float('lambda_l1', 0.0, 10.0),
#         'lambda_l2': trial.suggest_float('lambda_l2', 0.0, 10.0),
#     }
#     # 3‐fold stratified CV
#     skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     losses = []
#     for train_idx, val_idx in skf.split(X, y):
#         dtrain = lgb.Dataset(X[train_idx], label=y[train_idx])
#         dval   = lgb.Dataset(X[val_idx], label=y[val_idx])
#         gbm = lgb.train(
#             param, dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[
#                 lgb.early_stopping(stopping_rounds=50),
#                 lgb.log_evaluation(period=0)
#             ],
#         )
#         preds = gbm.predict(X[val_idx])
#         losses.append(log_loss(y[val_idx], preds))
#     return np.mean(losses)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=40, timeout=600)
# best_params = study.best_params
# print("Best params:", best_params)

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Bagged predictions with tuned params
# BAGS = 5
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# test_preds = np.zeros((X_test.shape[0], n_classes))

# for bag in range(BAGS):
#     print(f"Bag {bag+1}/{BAGS}")
#     seed = 1000 + bag
#     preds_oof = np.zeros_like(test_preds)

#     for train_idx, val_idx in skf.split(X, y):
#         dtrain = lgb.Dataset(X[train_idx], label=y[train_idx])
#         dval = lgb.Dataset(X[val_idx], label=y[val_idx])

#         gbm = lgb.train(
#             {**best_params, 'seed': seed, 'objective': 'multiclass', 'num_class': n_classes, 'metric': 'multi_logloss'},
#             dtrain,
#             num_boost_round=1000,
#             valid_sets=[dval],
#             callbacks=[lgb.early_stopping(50)],
#         )

#         preds_oof += gbm.predict(X_test) / skf.n_splits

#     test_preds += preds_oof / BAGS

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Create Submission
# top3 = np.argsort(test_preds, axis=1)[:, -3:][:, ::-1]
# labels = le.inverse_transform(top3.ravel()).reshape(top3.shape)
# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in labels]
# })
# submission.to_csv('submission_optuna_lgbm_bag.csv', index=False)
# print("✅ submission_optuna_lgbm_bag.csv saved")


# Base + Meta Neural Net

In [ ]:
# # Full End‑to‑End Stacking Pipeline with Feature Engineering, Calibrated Base Models, and Neural Meta-Learner

# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold, train_test_split
# from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
# from sklearn.calibration import CalibratedClassifierCV
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Load data
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# test .rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Separate target
# le_target = LabelEncoder()
# train['target'] = le_target.fit_transform(train['Fertilizer Name'])

# # Feature columns
# NUM_COLS = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# CAT_COLS = ['Soil Type','Crop Type']

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Feature engineering

# # 2.1: Polynomial features (degree 2 interactions)
# poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
# poly_feats_train = poly.fit_transform(train[NUM_COLS])
# poly_feats_test  = poly.transform(test[NUM_COLS])
# poly_names = poly.get_feature_names_out(NUM_COLS)

# df_poly_train = pd.DataFrame(poly_feats_train, columns=poly_names, index=train.index)
# df_poly_test  = pd.DataFrame(poly_feats_test,  columns=poly_names, index=test.index)

# # 2.2: Target-mean encoding for categoricals
# soil_map = train.groupby('Soil Type')['target'].mean()
# crop_map = train.groupby('Crop Type')['target'].mean()
# train['Soil_TE'] = train['Soil Type'].map(soil_map)
# test['Soil_TE']  = test['Soil Type'].map(soil_map)
# train['Crop_TE'] = train['Crop Type'].map(crop_map)
# test['Crop_TE']  = test['Crop Type'].map(crop_map)

# # 2.3: Label encode categoricals for base models
# le_soil = LabelEncoder().fit(train['Soil Type'])
# le_crop = LabelEncoder().fit(train['Crop Type'])
# train['Soil_LE'] = le_soil.transform(train['Soil Type'])
# train['Crop_LE'] = le_crop.transform(train['Crop Type'])
# test['Soil_LE']  = le_soil.transform(test['Soil Type'])
# test['Crop_LE']  = le_crop.transform(test['Crop Type'])

# # 2.4: Assemble final numeric feature matrices
# BASE_NUMERIC = NUM_COLS + ['Soil_LE','Crop_LE','Soil_TE','Crop_TE']
# X_base_train = train[BASE_NUMERIC].copy()
# X_base_test  = test[BASE_NUMERIC].copy()

# # Combine with polynomial features
# X_base_train = pd.concat([X_base_train, df_poly_train], axis=1)
# X_base_test  = pd.concat([X_base_test,  df_poly_test], axis=1)

# # Remove duplicate columns
# X_base_train = X_base_train.loc[:, ~X_base_train.columns.duplicated()]
# X_base_test  = X_base_test.loc[:, ~X_base_test.columns.duplicated()]

# y = train['target'].values
# n_classes = len(le_target.classes_)

# # Optional: scale numeric features for neural meta model later
# scaler = StandardScaler().fit(X_base_train)
# X_scaled_train = scaler.transform(X_base_train)
# X_scaled_test  = scaler.transform(X_base_test)

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Generate calibrated OOF predictions for base models

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# oof_preds = {
#     'lgb': np.zeros((len(train), n_classes)),
#     'xgb': np.zeros((len(train), n_classes)),
#     'cat': np.zeros((len(train), n_classes))
# }
# test_preds = {
#     'lgb': np.zeros((len(test), n_classes)),
#     'xgb': np.zeros((len(test), n_classes)),
#     'cat': np.zeros((len(test), n_classes))
# }

# for fold, (tr_idx, val_idx) in enumerate(skf.split(X_base_train, y), 1):
#     X_tr, X_val = X_base_train.iloc[tr_idx], X_base_train.iloc[val_idx]
#     y_tr, y_val = y[tr_idx], y[val_idx]
    
#     # LightGBM
#     m_lgb = lgb.LGBMClassifier(random_state=fold)
#     m_lgb.fit(X_tr, y_tr)
#     cal_lgb = CalibratedClassifierCV(m_lgb, cv=3, method='sigmoid')
#     cal_lgb.fit(X_val, y_val)
#     oof_preds['lgb'][val_idx] = cal_lgb.predict_proba(X_val)
#     test_preds['lgb'] += cal_lgb.predict_proba(X_base_test) / skf.n_splits

#     # XGBoost
#     m_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=fold)
#     m_xgb.fit(X_tr, y_tr)
#     cal_xgb = CalibratedClassifierCV(m_xgb, cv=3, method='sigmoid')
#     cal_xgb.fit(X_val, y_val)
#     oof_preds['xgb'][val_idx] = cal_xgb.predict_proba(X_val)
#     test_preds['xgb'] += cal_xgb.predict_proba(X_base_test) / skf.n_splits

#     # CatBoost
#     m_cat = cb.CatBoostClassifier(verbose=0, random_state=fold)
#     m_cat.fit(X_tr, y_tr)
#     cal_cat = CalibratedClassifierCV(m_cat, cv=3, method='sigmoid')
#     cal_cat.fit(X_val, y_val)
#     oof_preds['cat'][val_idx] = cal_cat.predict_proba(X_val)
#     test_preds['cat'] += cal_cat.predict_proba(X_base_test) / skf.n_splits

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Prepare meta‑model inputs

# # OOF stacked probabilities
# X_meta_oof  = np.hstack([oof_preds[m] for m in ['lgb','xgb','cat']])
# X_meta_test = np.hstack([test_preds[m]   for m in ['lgb','xgb','cat']])

# # Also include scaled original features for meta model
# X_meta_oof  = np.hstack([X_meta_oof,  X_scaled_train])
# X_meta_test = np.hstack([X_meta_test, X_scaled_test])

# # Convert y to categorical for meta NN
# y_cat_oof = tf.keras.utils.to_categorical(y, num_classes=n_classes)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Build & train neural network meta‑learner

# input_dim = X_meta_oof.shape[1]
# inp = Input(shape=(input_dim,), name='meta_input')
# x = Dense(64, activation='relu')(inp)
# x = BatchNormalization()(x)
# x = Dropout(0.3)(x)
# x = Dense(32, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
# out = Dense(n_classes, activation='softmax')(x)

# meta_model = Model(inp, out)
# meta_model.compile(
#     optimizer=Adam(learning_rate=1e-3),
#     loss='categorical_crossentropy'
# )

# # Train/validation split for meta model
# X_tr_meta, X_val_meta, y_tr_meta, y_val_meta = train_test_split(
#     X_meta_oof, y_cat_oof, test_size=0.2, random_state=42, stratify=y
# )

# meta_model.fit(
#     X_tr_meta, y_tr_meta,
#     validation_data=(X_val_meta, y_val_meta),
#     epochs=20,
#     batch_size=512,
#     verbose=2
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 6️⃣ Predict final and build submission

# meta_probs = meta_model.predict(X_meta_test)
# top3_idx = np.argsort(meta_probs, axis=1)[:, -3:][:, ::-1]
# top3_labels = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# submission = pd.DataFrame({
#     'id': test['id'],
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })
# submission.to_csv('submission_stacked_nn.csv', index=False)
# print("✅ submission_stacked_nn.csv saved")


# Wide & Deep model in a TPUStrategy

In [ ]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten
# from tensorflow.keras.models import Model
# from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.model_selection import train_test_split

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Load & Preprocess Data
# df = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
# test_df = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")

# # Fix typo
# df.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test_df.rename(columns={'Temparature':'Temperature'}, inplace=True)

# # Encode target
# le_target = LabelEncoder()
# df['target'] = le_target.fit_transform(df['Fertilizer Name'])

# # Feature columns
# cat_cols  = ['Soil Type', 'Crop Type']
# cont_cols = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']

# # Label‑encode categoricals
# encoders = {}
# for col in cat_cols:
#     enc = LabelEncoder().fit(df[col])
#     df[col]      = enc.transform(df[col])
#     test_df[col] = enc.transform(test_df[col])
#     encoders[col] = enc

# # Scale continuous
# scaler = StandardScaler().fit(df[cont_cols])
# df[cont_cols]      = scaler.transform(df[cont_cols])
# test_df[cont_cols] = scaler.transform(test_df[cont_cols])

# # Train/validation split
# train_df, val_df = train_test_split(
#     df, test_size=0.2, stratify=df['target'], random_state=42
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Build & Compile Wide & Deep Model

# # Categorical inputs & embeddings
# inputs, embeds = [], []
# for col in cat_cols:
#     vocab = df[col].nunique()
#     inp = Input(shape=(1,), name=f"{col}_in")
#     emb = Embedding(input_dim=vocab+1, output_dim=4)(inp)
#     emb = Flatten()(emb)
#     inputs.append(inp)
#     embeds.append(emb)

# # Continuous input
# cont_input = Input(shape=(len(cont_cols),), name="cont_in")
# inputs.append(cont_input)

# # Wide branch
# wide = Dense(64, activation='relu')(cont_input)

# # Deep branch
# x = Concatenate()(embeds + [wide])
# x = Dense(128, activation='relu')(x)
# x = Dense(64, activation='relu')(x)
# output = Dense(len(le_target.classes_), activation='softmax')(x)

# model = Model(inputs, output)
# model.compile(
#     optimizer='adam',
#     loss='sparse_categorical_crossentropy',
#     metrics=['sparse_categorical_accuracy']
# )
# model.summary()

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Prepare tf.data Datasets for Training

# def make_dataset(df_, shuffle=False, batch_size=1024):
#     cat_arrays = [df_[col].values for col in cat_cols]
#     cont_array = df_[cont_cols].values
#     X = tuple(cat_arrays + [cont_array])
#     y = df_['target'].values
#     ds = tf.data.Dataset.from_tensor_slices((X, y))
#     if shuffle:
#         ds = ds.shuffle(buffer_size=len(df_))
#     return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# batch_size = 1024
# train_ds = make_dataset(train_df, shuffle=True,  batch_size=batch_size)
# val_ds   = make_dataset(val_df,   shuffle=False, batch_size=batch_size)

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Train the Model
# model.fit(
#     train_ds,
#     validation_data=val_ds,
#     epochs=20,
#     verbose=2
# )

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Predict & Build Submission (direct NumPy inputs)
# # Prepare numpy inputs in same order as model inputs
# test_inputs = [test_df[col].values for col in cat_cols] + [test_df[cont_cols].values]

# # Predict probabilities
# probs = model.predict(test_inputs, batch_size=batch_size, verbose=0)

# # Extract top‑3 predictions
# top3_idx    = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# top3_labels = le_target.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# # Create submission
# submission = pd.DataFrame({
#     "id": test_df["id"],
#     "Fertilizer Name": [' '.join(row) for row in top3_labels]
# })
# submission.to_csv("submission_wide_deep.csv", index=False)
# print("✅ submission_wide_deep.csv saved")


# Weighted Ensemble with Optuna‑Tuned Weights + Probability Calibration

In [ ]:
# !pip install pytorch_tabular --quiet

In [ ]:
# pip install "pytorch_tabular[extra]"


In [ ]:
# !pip install pytorch-tabular[extra] --quiet


In [ ]:
# # 📦 Imports
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.calibration import CalibratedClassifierCV
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb
# import optuna

# # ───────────────────────────────────────────────────────────────────────────────
# # 1️⃣ Load & Preprocess Data
# df_train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# df_test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix column typo
# df_train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# df_test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Encode target
# le = LabelEncoder()
# df_train['target'] = le.fit_transform(df_train['Fertilizer Name'])

# # Define features
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
# X = df_train[features]
# y = df_train['target']
# X_test = df_test[features]

# # ───────────────────────────────────────────────────────────────────────────────
# # 2️⃣ Stratified K-Fold Setup
# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
# n_classes = len(le.classes_)

# # Initialize prediction holders
# oof_lgb = np.zeros((len(X), n_classes))
# oof_xgb = np.zeros((len(X), n_classes))
# oof_cat = np.zeros((len(X), n_classes))
# test_lgb = np.zeros((len(X_test), n_classes))
# test_xgb = np.zeros((len(X_test), n_classes))
# test_cat = np.zeros((len(X_test), n_classes))

# # ───────────────────────────────────────────────────────────────────────────────
# # 3️⃣ Train each model with calibration
# for train_idx, val_idx in skf.split(X, y):
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # ✅ LightGBM + Calibration
#     lgb_model = lgb.LGBMClassifier()
#     lgb_model.fit(X_tr, y_tr)
#     cal_lgb = CalibratedClassifierCV(base_estimator=lgb_model, cv=5, method='sigmoid')
#     cal_lgb.fit(X_val, y_val)
#     oof_lgb[val_idx] = cal_lgb.predict_proba(X_val)
#     test_lgb += cal_lgb.predict_proba(X_test) / n_splits

#     # ✅ XGBoost + Calibration
#     xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
#     xgb_model.fit(X_tr, y_tr)
#     cal_xgb = CalibratedClassifierCV(base_estimator=xgb_model, cv=5, method='sigmoid')
#     cal_xgb.fit(X_val, y_val)
#     oof_xgb[val_idx] = cal_xgb.predict_proba(X_val)
#     test_xgb += cal_xgb.predict_proba(X_test) / n_splits

#     # ✅ CatBoost + Calibration
#     cat_model = cb.CatBoostClassifier(verbose=0)
#     cat_model.fit(X_tr, y_tr)
#     cal_cat = CalibratedClassifierCV(base_estimator=cat_model, cv=5, method='sigmoid')
#     cal_cat.fit(X_val, y_val)
#     oof_cat[val_idx] = cal_cat.predict_proba(X_val)
#     test_cat += cal_cat.predict_proba(X_test) / n_splits

# # ───────────────────────────────────────────────────────────────────────────────
# # 4️⃣ Use Optuna for best ensemble weights
# def objective(trial):
#     w1 = trial.suggest_float("w_lgb", 0.0, 1.0)
#     w2 = trial.suggest_float("w_xgb", 0.0, 1.0)
#     w3 = trial.suggest_float("w_cat", 0.0, 1.0)
#     total = w1 + w2 + w3
#     w1, w2, w3 = w1 / total, w2 / total, w3 / total
#     oof_combined = w1 * oof_lgb + w2 * oof_xgb + w3 * oof_cat
#     return log_loss(y, oof_combined)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=30)

# best = study.best_params
# total = best['w_lgb'] + best['w_xgb'] + best['w_cat']
# w_lgb = best['w_lgb'] / total
# w_xgb = best['w_xgb'] / total
# w_cat = best['w_cat'] / total
# print("✅ Optimized Weights — LGB:", w_lgb, "XGB:", w_xgb, "CAT:", w_cat)

# # ───────────────────────────────────────────────────────────────────────────────
# # 5️⃣ Final Prediction & Submission
# test_final = w_lgb * test_lgb + w_xgb * test_xgb + w_cat * test_cat

# top3_idx = np.argsort(test_final, axis=1)[:, -3:][:, ::-1]
# top3_labels = le.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# submission = pd.DataFrame({
#     "id": df_test["id"],
#     "Fertilizer Name": [" ".join(row) for row in top3_labels]
# })
# submission.to_csv("submission_ensemble.csv", index=False)
# print("🎉 submission_ensemble.csv saved successfully!")

# Pseudo-Labeled Stacked Ensemble model

In [ ]:
# # 1. Data Loading & Basic Preprocessing
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb

# # Load and fix typo
# train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')
# train.rename(columns={'Temparature':'Temperature'}, inplace=True)
# test.rename(columns={'Temparature':'Temperature'},   inplace=True)

# # Encode target and generate simple encodings
# le_target = LabelEncoder()
# train['Target'] = le_target.fit_transform(train['Fertilizer Name'])

# for col in ['Soil Type','Crop Type']:
#     le = LabelEncoder()
#     train[col] = le.fit_transform(train[col])
#     test[col]  = le.transform(test[col])

# # Feature set
# features = ['Temperature','Humidity','Moisture','Nitrogen',
#             'Potassium','Phosphorous','Soil Type','Crop Type']
# X = train[features]
# y = train['Target']
# X_test = test[features]
# test_ids = test['id']


# # ==== 2. First‑Stage Stacking: OOF + Test Predictions ====
# n_classes = len(le_target.classes_)
# n_train = len(train)
# n_test  = len(test)

# # Prepare arrays
# oof_preds = {name: np.zeros((n_train, n_classes)) for name in ['lgb','xgb','cat']}
# test_preds = {name: np.zeros((n_test, n_classes))  for name in ['lgb','xgb','cat']}

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for tr_idx, val_idx in skf.split(X, y):
#     X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
    
#     # LightGBM
#     m = lgb.LGBMClassifier()
#     m.fit(X_tr, y_tr)
#     oof_preds['lgb'][val_idx] = m.predict_proba(X_val)
#     test_preds['lgb'] += m.predict_proba(X_test) / skf.n_splits
    
#     # XGBoost
#     m = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
#     m.fit(X_tr, y_tr)
#     oof_preds['xgb'][val_idx] = m.predict_proba(X_val)
#     test_preds['xgb'] += m.predict_proba(X_test) / skf.n_splits
    
#     # CatBoost
#     m = cb.CatBoostClassifier(verbose=0)
#     m.fit(X_tr, y_tr)
#     oof_preds['cat'][val_idx] = m.predict_proba(X_val)
#     test_preds['cat'] += m.predict_proba(X_test) / skf.n_splits

# # Stack meta‑features
# X_meta = np.hstack([oof_preds[m] for m in ['lgb','xgb','cat']])
# X_test_meta = np.hstack([test_preds[m] for m in ['lgb','xgb','cat']])

# # ==== 3. Pseudo‑Labeling ====
# # Get high‑confidence test labels (threshold 0.90)
# conf_thresh = 0.90
# pseudo_idxs = []
# pseudo_X = []
# pseudo_y = []

# # For each test row, if top‑1 prob > thresh, add to pseudo
# probs = X_test_meta.reshape(n_test, -1)  # meta‑features same shape
# # But we need original test_preds for labeling
# avg_probs = (test_preds['lgb'] + test_preds['xgb'] + test_preds['cat'])/3
# top1 = np.max(avg_probs, axis=1)
# pred1 = np.argmax(avg_probs, axis=1)
# for i in range(n_test):
#     if top1[i] > conf_thresh:
#         pseudo_idxs.append(i)
#         pseudo_X.append(X_test_meta[i])
#         pseudo_y.append(pred1[i])

# # Augment meta-training set
# if pseudo_X:
#     X_meta = np.vstack([X_meta, np.array(pseudo_X)])
#     y_meta = np.concatenate([y, np.array(pseudo_y)])
# else:
#     y_meta = y

# # ==== 4. Meta‑Model Training on Augmented Data ====
# meta = LogisticRegression(multi_class='multinomial', max_iter=1000)
# meta.fit(X_meta, y_meta)

# # ==== 5. Final Predictions & Submission ====
# final_probs = meta.predict_proba(X_test_meta)
# top3 = np.argsort(final_probs, axis=1)[:, -3:][:, ::-1]

# submission = pd.DataFrame({
#     'id': test_ids,
#     'Fertilizer Name': [
#         ' '.join(le_target.inverse_transform(row)) for row in top3
#     ]
# })
# submission.to_csv('submission_pseudo_stacked.csv', index=False)


# lightgbm, xgboost, and catboost

In [ ]:
# Install required libraries for advanced ML models
!pip install lightgbm xgboost catboost --quiet

## With Folds

In [ ]:
# # 1. Data Loading and Preprocessing ====
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.linear_model import LogisticRegression

# # Base model libraries
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb

# # Load datasets
# df_train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# df_test  = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix column typo
# df_train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# df_test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Label‑encode target for numeric mapping
# le_target = LabelEncoder()
# df_train['Target'] = le_target.fit_transform(df_train['Fertilizer Name'])

# # Numeric Target Encoding (mean target label) for Soil and Crop
# soil_mean = df_train.groupby('Soil Type')['Target'].mean()
# crop_mean = df_train.groupby('Crop Type')['Target'].mean()

# df_train['Soil_TE'] = df_train['Soil Type'].map(soil_mean)
# df_test['Soil_TE']  = df_test['Soil Type'].map(soil_mean)

# df_train['Crop_TE'] = df_train['Crop Type'].map(crop_mean)
# df_test['Crop_TE']  = df_test['Crop Type'].map(crop_mean)

# # Label encode categorical features
# le_soil = LabelEncoder()
# le_crop = LabelEncoder()

# df_train['Soil'] = le_soil.fit_transform(df_train['Soil Type'])
# df_test['Soil']  = le_soil.transform(df_test['Soil Type'])

# df_train['Crop'] = le_crop.fit_transform(df_train['Crop Type'])
# df_test['Crop']  = le_crop.transform(df_test['Crop Type'])

# # Feature list
# features = ['Temperature','Humidity','Moisture','Nitrogen',
#             'Potassium','Phosphorous','Soil','Crop','Soil_TE','Crop_TE']

# X = df_train[features].reset_index(drop=True)
# y = df_train['Target'].reset_index(drop=True)
# X_test = df_test[features].reset_index(drop=True)
# test_ids = df_test['id']

# # ==== 2. Prepare Arrays for Stacking ====
# n_classes = len(le_target.classes_)
# n_train = X.shape[0]
# n_test  = X_test.shape[0]

# oof_lgb = np.zeros((n_train, n_classes))
# oof_xgb = np.zeros((n_train, n_classes))
# oof_cat = np.zeros((n_train, n_classes))

# test_lgb = np.zeros((n_test,  n_classes))
# test_xgb = np.zeros((n_test,  n_classes))
# test_cat = np.zeros((n_test,  n_classes))

# # ==== 3. Generate OOF Predictions for Base Learners ====
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for train_idx, val_idx in skf.split(X, y):
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
#     # LightGBM
#     m_lgb = lgb.LGBMClassifier()
#     m_lgb.fit(X_tr, y_tr)
#     oof_lgb[val_idx] = m_lgb.predict_proba(X_val)
#     test_lgb += m_lgb.predict_proba(X_test) / skf.n_splits
    
#     # XGBoost
#     m_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
#     m_xgb.fit(X_tr, y_tr)
#     oof_xgb[val_idx] = m_xgb.predict_proba(X_val)
#     test_xgb += m_xgb.predict_proba(X_test) / skf.n_splits
    
#     # CatBoost
#     m_cat = cb.CatBoostClassifier(verbose=0)
#     m_cat.fit(X_tr, y_tr)
#     oof_cat[val_idx] = m_cat.predict_proba(X_val)
#     test_cat += m_cat.predict_proba(X_test) / skf.n_splits

# # Stack OOF predictions as new features
# X_meta_train = np.hstack([oof_lgb, oof_xgb, oof_cat])
# X_meta_test  = np.hstack([test_lgb, test_xgb, test_cat])

# # ==== 4. Train Meta‑Model on OOF Predictions ====
# meta_model = LogisticRegression(multi_class='multinomial', max_iter=1000)
# meta_model.fit(X_meta_train, y)

# # ==== 5. Predict with Meta‑Model and Prepare Submission ====
# meta_preds = meta_model.predict_proba(X_meta_test)
# top3 = np.argsort(meta_preds, axis=1)[:, -3:][:, ::-1]

# submission = pd.DataFrame({
#     'id': test_ids,
#     'Fertilizer Name': [' '.join(le_target.inverse_transform(row)) for row in top3]
# })
# submission.to_csv('submission_stacked.csv', index=False)


# with normal

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import LabelEncoder
# import lightgbm as lgb
# import xgboost as xgb
# import catboost as cb

# # Load data
# df_train = pd.read_csv('/kaggle/input/playground-series-s5e6/train.csv')
# df_test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

# # Fix typo
# df_train.rename(columns={'Temparature': 'Temperature'}, inplace=True)
# df_test.rename(columns={'Temparature': 'Temperature'}, inplace=True)

# # Target Encoding
# soil_map = df_train.groupby('Soil Type')['Fertilizer Name'].agg(lambda x: x.value_counts().idxmax())
# crop_map = df_train.groupby('Crop Type')['Fertilizer Name'].agg(lambda x: x.value_counts().idxmax())
# df_train['Soil_TE'] = df_train['Soil Type'].map(soil_map)
# df_test['Soil_TE'] = df_test['Soil Type'].map(soil_map)
# df_train['Crop_TE'] = df_train['Crop Type'].map(crop_map)
# df_test['Crop_TE'] = df_test['Crop Type'].map(crop_map)

# # Label Encoding
# le_soil = LabelEncoder()
# le_crop = LabelEncoder()
# le_target = LabelEncoder()
# df_train['Soil'] = le_soil.fit_transform(df_train['Soil Type'])
# df_test['Soil'] = le_soil.transform(df_test['Soil Type'])
# df_train['Crop'] = le_crop.fit_transform(df_train['Crop Type'])
# df_test['Crop'] = le_crop.transform(df_test['Crop Type'])
# df_train['Target'] = le_target.fit_transform(df_train['Fertilizer Name'])

# # Features
# features = ['Temperature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Soil', 'Crop']
# X = df_train[features]
# y = df_train['Target']
# X_test = df_test[features]

# # Predictions
# lgb_preds = np.zeros((X_test.shape[0], len(le_target.classes_)))
# xgb_preds = np.zeros((X_test.shape[0], len(le_target.classes_)))
# cat_preds = np.zeros((X_test.shape[0], len(le_target.classes_)))

# # Cross-validation
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for train_idx, val_idx in skf.split(X, y):
#     X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
#     y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

#     # LightGBM
#     model_lgb = lgb.LGBMClassifier()
#     model_lgb.fit(X_tr, y_tr)
#     lgb_preds += model_lgb.predict_proba(X_test) / skf.n_splits

#     # XGBoost
#     model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
#     model_xgb.fit(X_tr, y_tr)
#     xgb_preds += model_xgb.predict_proba(X_test) / skf.n_splits

#     # CatBoost
#     model_cat = cb.CatBoostClassifier(verbose=0)
#     model_cat.fit(X_tr, y_tr)
#     cat_preds += model_cat.predict_proba(X_test) / skf.n_splits

# # Ensemble
# avg_preds = (lgb_preds + xgb_preds + cat_preds) / 3
# top_3 = np.argsort(avg_preds, axis=1)[:, -3:][:, ::-1]

# # Submission
# submission = pd.DataFrame()
# submission['id'] = df_test['id']
# submission['Fertilizer Name'] = [' '.join(le_target.inverse_transform(row)) for row in top_3]
# submission.to_csv('submission.csv', index=False)
# submission.head()


# Model ZOO with DF_train

In [ ]:
# # 📦 Import Libraries
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import log_loss
# import lightgbm as lgb
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier

# # 📁 Load Data
# df_train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
# df_test  = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")

# # 🧹 Drop ID column and separate target
# df_train = df_train.drop(columns=['id'])
# y = df_train['Fertilizer Name']
# df_train = df_train.drop(columns=['Fertilizer Name'])
# test_ids = df_test['id']
# df_test  = df_test.drop(columns=['id'])

# # 🔠 Label‑encode categorical features
# for col in ['Soil Type', 'Crop Type']:
#     le = LabelEncoder()
#     df_train[col] = le.fit_transform(df_train[col])
#     df_test[col]  = le.transform(df_test[col])

# # 🔠 Encode target
# target_le = LabelEncoder()
# y_enc = target_le.fit_transform(y)

# # ⚙️ Create interaction feature: Soil_Crop
# df_train['Soil_Crop'] = df_train['Soil Type'].astype(str) + "_" + df_train['Crop Type'].astype(str)
# df_test['Soil_Crop']  = df_test['Soil Type'].astype(str)  + "_" + df_test['Crop Type'].astype(str)

# # ➕ Polynomial Features
# num_cols = ['Temparature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
# df_train_poly = pd.DataFrame(poly.fit_transform(df_train[num_cols]),
#                              columns=poly.get_feature_names_out(num_cols),
#                              index=df_train.index)
# df_test_poly  = pd.DataFrame(poly.transform(df_test[num_cols]),
#                              columns=poly.get_feature_names_out(num_cols),
#                              index=df_test.index)

# # 📈 Group‑based mean stats by Soil_Crop
# group_stats = df_train.groupby('Soil_Crop')[num_cols].mean().add_prefix('SC_mean_')
# df_train = df_train.join(group_stats, on='Soil_Crop')
# df_test  = df_test.join(group_stats, on='Soil_Crop')

# # 🧱 Final feature sets (drop the text interaction column, merge polys)
# df_train_final = pd.concat([df_train.drop(columns=['Soil_Crop']), df_train_poly], axis=1)
# df_test_final  = pd.concat([df_test.drop(columns=['Soil_Crop']), df_test_poly],  axis=1)

# #to prevent duplicate columns
# df_train_final = df_train_final.loc[:, ~df_train_final.columns.duplicated()]
# df_test_final  = df_test_final.loc[:, ~df_test_final.columns.duplicated()]

# # ⚒️ Define models (no TabNet)
# models = {
#     'LightGBM': lgb.LGBMClassifier(
#         objective='multiclass', num_class=len(target_le.classes_),
#         learning_rate=0.05, num_leaves=31, subsample=0.8,
#         colsample_bytree=0.8, n_estimators=300
#     ),
#     'XGBoost': XGBClassifier(
#         objective='multi:softprob', num_class=len(target_le.classes_),
#         learning_rate=0.1, max_depth=6, subsample=0.8,
#         colsample_bytree=0.8, n_estimators=300,
#         use_label_encoder=False, eval_metric='mlogloss'
#     ),
#     'CatBoost': CatBoostClassifier(
#         loss_function='MultiClass', learning_rate=0.1,
#         depth=6, iterations=300, verbose=0
#     )
# }

# # 🔁 Cross‑validation
# cv_scores = {name: [] for name in models}
# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for fold, (train_idx, val_idx) in enumerate(kf.split(df_train_final, y_enc), 1):
#     X_tr, X_val = df_train_final.iloc[train_idx], df_train_final.iloc[val_idx]
#     y_tr, y_val = y_enc[train_idx], y_enc[val_idx]

#     print(f"\n🧪 Fold {fold}")
#     for name, model in models.items():
#         print(f"▶ Training {name}…", end=' ')
#         model.fit(X_tr, y_tr)
#         preds = model.predict_proba(X_val)
#         loss = log_loss(y_val, preds)
#         cv_scores[name].append(loss)
#         print(f"LogLoss = {loss:.4f}")

# # 📊 CV Summary
# print("\n🔚 Cross‑Validation Results:")
# for name, scores in cv_scores.items():
#     print(f"{name:>8}: Mean = {np.mean(scores):.4f} ± {np.std(scores):.4f}")

# # ───────────────────────────────────────────────────────────────────────────────
# # ▶ Final training & submission (example with XGBoost here)

# best_model = models['XGBoost']
# best_model.fit(df_train_final, y_enc)

# probs_test = best_model.predict_proba(df_test_final)
# top3_idx   = np.argsort(probs_test, axis=1)[:, -3:][:, ::-1]
# top3_lab   = target_le.inverse_transform(top3_idx.ravel()).reshape(top3_idx.shape)

# submission = pd.DataFrame({
#     'id': test_ids,
#     'Fertilizer Name': [' '.join(row) for row in top3_lab]
# })
# submission.to_csv('submission.csv', index=False)
# print("✅ submission.csv saved!")


# Light XGB model continue

In [ ]:
# # 1. Drop the ID column
# df_train = df_train.drop(columns=['id'])
# df_test  = df_test .drop(columns=['id'])

# # 2. One‑hot encode Soil Type & Crop Type
# df_train = pd.get_dummies(df_train, columns=['Soil Type', 'Crop Type'])
# df_test  = pd.get_dummies(df_test,  columns=['Soil Type', 'Crop Type'])

# # Align train/test columns (in case some categories appear only in one set)
# df_test = df_test.reindex(columns=df_train.columns.drop('Fertilizer Name'), fill_value=0)

# # 3. Encode the target variable
# le_target = LabelEncoder()
# df_train['Fertilizer Label'] = le_target.fit_transform(df_train['Fertilizer Name'])

# # Now drop the original target string column
# df_train = df_train.drop(columns=['Fertilizer Name'])

# # 4. (Optional) Scale numeric features if you plan to use non-tree models
# # from sklearn.preprocessing import StandardScaler
# # num_cols = ['Temperature','Humidity','Moisture','Nitrogen','Potassium','Phosphorous']
# # scaler = StandardScaler()
# # df_train[num_cols] = scaler.fit_transform(df_train[num_cols])
# # df_test[num_cols]  = scaler.transform(df_test[num_cols])

# # Final feature/label splits
# X = df_train.drop(columns=['Fertilizer Label'])
# y = df_train['Fertilizer Label']


In [ ]:
# print("X shape:", X.shape)
# print("y shape:", y.shape)


In [ ]:
# pd.Series(y).value_counts(normalize=True).head(10)


# LGBM model

In [ ]:
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split

# # 1. Split the data
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # 2. Convert to LightGBM datasets
# train_data = lgb.Dataset(X_train, label=y_train)
# val_data   = lgb.Dataset(X_val, label=y_val)

# # 3. Define parameters
# params = {
#     'objective': 'multiclass',
#     'num_class': y.nunique(),
#     'metric': 'multi_logloss',
#     'learning_rate': 0.1,
#     'verbosity': -1
# }

# # 4. Train with callbacks for early stopping
# model = lgb.train(
#     params,
#     train_data,
#     valid_sets=[val_data],                      # Must pass validation set
#     callbacks=[
#         lgb.early_stopping(stopping_rounds=20),
#         lgb.log_evaluation(period=10)           # Show logs every 10 rounds
#     ],
#     num_boost_round=500                         # Max rounds
# )


In [ ]:
# # Step 1: Predict probabilities for each fertilizer class
# probs = model.predict(df_test)

# # Step 2: Get top 3 class indices with highest probability
# top3 = np.argsort(probs, axis=1)[:, -3:][:, ::-1]  # [:, ::-1] reverses to get descending order

# # Step 3: Convert numeric predictions back to original fertilizer names
# top3_labels = le_target.inverse_transform(top3.ravel()).reshape(top3.shape)

# # Step 4: Load original test file to get 'id' column
# df_test_original = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")  # use your path

# # Step 5: Create submission DataFrame
# submission = pd.DataFrame({
#     'id': df_test_original['id'],
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })

# # Step 6: Save to CSV in correct format
# submission.to_csv('submission.csv', index=False)


# New model

In [ ]:
# import matplotlib.pyplot as plt
# import lightgbm as lgb

# # Plot feature importance
# lgb.plot_importance(model, max_num_features=15, importance_type='gain', figsize=(10, 6))
# plt.title("Top 15 Important Features")
# plt.grid(True)
# plt.tight_layout()
# plt.show()


In [ ]:
# from xgboost import XGBClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import log_loss
# import numpy as np

# # Encode target labels again (just in case)
# le = LabelEncoder()
# y_encoded = le.fit_transform(y)

# # Train/val split
# X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

# # Train XGBoost
# xgb_model = XGBClassifier(
#     objective='multi:softprob',
#     num_class=len(le.classes_),
#     learning_rate=0.1,
#     max_depth=6,
#     n_estimators=300,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     use_label_encoder=False,
#     eval_metric='mlogloss'
# )
# xgb_model.fit(X_train, y_train)

# # Predict and generate top-3
# xgb_probs = xgb_model.predict_proba(df_test)
# top3_xgb = np.argsort(xgb_probs, axis=1)[:, -3:][:, ::-1]
# top3_xgb_labels = le.inverse_transform(top3_xgb.ravel()).reshape(top3_xgb.shape)

# # Prepare submission
# submission_xgb = pd.DataFrame({
#     'id': df_test_original['id'],
#     'Fertilizer Name': [' '.join(row) for row in top3_xgb_labels]
# })
# submission_xgb.to_csv('submission_xgb.csv', index=False)


In [ ]:
# params = {
#     'objective': 'multiclass',
#     'num_class': y.nunique(),
#     'metric': 'multi_logloss',
#     'learning_rate': 0.05,
#     'max_depth': 8,
#     'num_leaves': 31,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'verbosity': -1
# }
# # You can now train like before using this improved param set.


In [ ]:
# import optuna

# def objective(trial):
#     param = {
#         'objective': 'multiclass',
#         'num_class': y.nunique(),
#         'metric': 'multi_logloss',
#         'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.2),
#         'num_leaves': trial.suggest_int("num_leaves", 20, 100),
#         'max_depth': trial.suggest_int("max_depth", 3, 15),
#         'subsample': trial.suggest_float("subsample", 0.5, 1.0),
#         'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1.0),
#         'verbosity': -1
#     }

#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
#     dtrain = lgb.Dataset(X_train, label=y_train)
#     dval = lgb.Dataset(X_val, label=y_val)

#     model = lgb.train(param, dtrain, valid_sets=[dval], callbacks=[lgb.early_stopping(10)], num_boost_round=300)
#     preds = model.predict(X_val)
#     return log_loss(y_val, preds)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=30)

# # Best params
# print("Best parameters: ", study.best_params)


# New XGB code

In [ ]:
# '''
# XGBoost Pipeline for Fertilizer Recommendation (Kaggle Playground S5E6)
# Step-by-step code with detailed comments
# '''

# # STEP 1: Import necessary libraries
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import log_loss
# from xgboost import XGBClassifier

# # STEP 2: Load the data
# # Replace the paths below with your actual Kaggle dataset paths
# train_path = '/kaggle/input/playground-series-s5e6/train.csv'
# test_path  = '/kaggle/input/playground-series-s5e6/test.csv'

# df_train = pd.read_csv(train_path)
# df_test  = pd.read_csv(test_path)

# # Quick preview
# print(df_train.head())
# print(df_train.info())
# print("Unique fertilizers:", df_train['Fertilizer Name'].nunique())

# # STEP 3: Basic preprocessing
# # 3.1 Drop 'id' columns (not predictive)
# df_train = df_train.drop(columns=['id'])
# df_test_ids = df_test['id'].copy()
# df_test  = df_test.drop(columns=['id'])

# # 3.2 Create interaction feature: Soil_Crop
# # This captures joint effect of soil and crop combination
# for df in [df_train, df_test]:
#     df['Soil_Crop'] = df['Soil Type'] + '_' + df['Crop Type']

# # STEP 4: Encoding categorical features
# # Use LabelEncoder for simplicity on all categorical predictors
# cat_cols = ['Soil Type', 'Crop Type', 'Soil_Crop']
# le_dict = {}
# for col in cat_cols:
#     le = LabelEncoder()
#     df_train[col] = le.fit_transform(df_train[col])
#     df_test[col]  = le.transform(df_test[col])
#     le_dict[col] = le  # store encoder if needed later

# # Encode the target variable
# le_target = LabelEncoder()
# train_labels = le_target.fit_transform(df_train['Fertilizer Name'])
# # Drop original target column
# df_train = df_train.drop(columns=['Fertilizer Name'])

# # STEP 5: Prepare feature matrix X and label vector y
# X = df_train.copy()
# y = train_labels
# X_test = df_test.copy()

# # STEP 6: Train-validation split for local evaluation
# X_train, X_val, y_train, y_val = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# # STEP 7: Train XGBoost model with early stopping
# xgb_model = XGBClassifier(
#     objective='multi:softprob',       # multiclass classification with probabilities
#     num_class=len(le_target.classes_),
#     learning_rate=0.1,                # step size shrinkage
#     max_depth=6,                      # maximum tree depth
#     n_estimators=300,                 # number of trees
#     subsample=0.8,                    # subsample ratio of training instances
#     colsample_bytree=0.8,             # subsample ratio of columns when constructing each tree
#     use_label_encoder=False,          # disable default label encoder
#     eval_metric='mlogloss',           # evaluation metric
#     random_state=42
# )

# # Fit with early stopping on validation data
# xgb_model.fit(
#     X_train, y_train,
#     eval_set=[(X_val, y_val)],
#     early_stopping_rounds=20,
#     verbose=True
# )

# # STEP 8: Plot feature importance
# plt.figure(figsize=(10, 6))
# importance = xgb_model.feature_importances_
# features = X.columns
# indices = np.argsort(importance)[-15:]  # top 15 features
# plt.barh(range(len(indices)), importance[indices], align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.title('Top 15 XGBoost Feature Importances')
# plt.xlabel('Feature importance')
# plt.tight_layout()
# plt.show()

# # STEP 9: Generate top-3 predictions for test set
# # 9.1 Predict class probabilities
# probs = xgb_model.predict_proba(X_test)
# # 9.2 Select top 3 indices per sample
# top3_indices = np.argsort(probs, axis=1)[:, -3:][:, ::-1]
# # 9.3 Map indices back to fertilizer names
# top3_labels = le_target.inverse_transform(top3_indices.ravel()).reshape(top3_indices.shape)

# # STEP 10: Prepare and save submission
# submission = pd.DataFrame({
#     'id': df_test_ids,
#     'Fertilizer Name': [' '.join(row) for row in top3_labels]
# })

# submission.to_csv('submission_xgb.csv', index=False)
# print("Submission file 'submission_xgb.csv' created.")
